In [1]:
import gc

import numpy as np
import pandas as pd
from datetime import datetime
import os
from pathlib import Path
import re
import phonenumbers
from multiprocessing import Pool,Process,Queue,Manager,resource_tracker
from collections import defaultdict
import json
from sklearn import metrics
from datetime import datetime, timedelta


In [2]:
def normalize_phone(phone,country_id):
    # 去掉phone中的非数字和非字母
    def phone_normalize(phone):
        # 清洗+00593964132978该类手机号
        # print(phone)
        if len(phone) == 0 or phone == None:
            return 0, '', ''
        if phone[0] == '+':
            phone = '+' + phone.replace('+', '').lstrip('0')
            length = len(phone) - 1
        else:
            phone = phone.lstrip('0')
            length = len(phone)
        # 初始化
        is_vaild, country_code, national_phone = 0, '', phone.lstrip('+')

        # 若手机号长度小于7（最短手机号），则返回
        if length < 7:
            return str(national_phone)

        try:
            if phone.startswith('+'):
                parse_info = phonenumbers.parse(phone)
            else:
                parse_info = phonenumbers.parse(phone, country_id.upper())

            # print(phonenumbers.is_valid_number(parse_info))
            is_vaild = 1 if phonenumbers.is_valid_number(parse_info) else 0
            if is_vaild:
                # country_code = parse_info.country_code
                national_phone = parse_info.national_number
                return str(national_phone)
        except Exception:
            pass
        return str(national_phone)

    phone = str(phone)
    try:
        if len(phone) == 0 or phone == None:
            return ''
        if phone[0] == '+':
            phone = '+'+''.join(filter(str.isalnum, phone)).lower()
        else:
            phone = ''.join(filter(str.isalnum, phone)).lower()
    except Exception as e:
        pass
        #print(phone)
        # print(e)

    # print(phone)
    # 如果phone包含字母，返回0,'',''
    if any(c.isalpha() for c in phone):
        return phone.replace('+','')
    # 如果phone为纯数字，返回phone_normalize处理结果
    return phone_normalize(phone)

In [3]:
def numerical_univerate(df, var_name, target, labels=None, bins=10, bins_edges=None, lamb=0.001, max_categories=30):
    """
    单变量分箱函数
    :param df:  dataframe
    :param var_name:  变量名
    :param target:  目标变量
    :param labels:  分箱标签
    :param bins:  分箱数
    :param bins_edges:  分箱边界
    :param lamb:  lamb
    :param max_categories:  类别特征最大类别数
    :return: 
    """
    new_col = var_name + "_bin"  
    if var_name not in df.columns:
        print(f"{var_name} not in df.columns")
        return None, None
    if df[var_name].dtype.name in ["category", "object", "dtype('O')"]:
        # 处理分类特征  
        value_counts = df[var_name].value_counts()  
        top_categories = value_counts.head(max_categories).index  
        df[new_col] = df[var_name].apply(lambda x: x if x in top_categories else 'Other')  
    else:  
        # 处理连续型特征  
        if bins_edges is None:  
            df[new_col], bin_edges = pd.qcut(df[var_name], bins, labels=labels, retbins=True, duplicates='drop')
        else:
            df[new_col] = pd.cut(df[var_name], bins_edges, labels=labels, include_lowest=True)
    bin_edges = df[new_col].value_counts().index.tolist()
    # bin_edges排序
    bin_edges.sort()

    dti = pd.crosstab(df[new_col], df[target]).sort_values(by=new_col,ascending=False)  
    # print(dti)  
    dti.rename(  
        {1: "positive",0: "negative"},  
        axis=1,  
        inplace=True,  
    )  
    if 'positive' not in dti.columns:
        return None, bin_edges
    if 'negative' not in dti.columns:
        return None, bin_edges
        
    # spearman_corr = df[var_name].corr(df[target],method='spearman')  
    dti["positive"] = dti["positive"].astype(int)  
    dti["negative"] = dti["negative"].astype(int)  
    p_t = dti["positive"].sum()  
    n_t = dti["negative"].sum()  
    t_t = p_t + n_t  
    r_t = p_t / t_t  
    dti["total"] = dti["positive"] + dti["negative"]  
    dti["total_rate"] = dti["total"] / t_t  
    dti["positive_rate"] = dti["positive"] / dti["total"]  
  
    dti["negative_cum"] = dti["negative"].cumsum()  
    dti["positive_cum"] = dti["positive"].cumsum()  
  
    dti['positive_rate_cum'] = dti["positive_cum"]/(dti["negative_cum"]+ dti["positive_cum"])  
  
    dti["woe"] = np.log(  
        ((dti["negative"] / n_t) + lamb) / ((dti["positive"] / p_t) + lamb)  
    )  
  
    dti["LIFT"] = dti["positive_rate"] / r_t  
    dti["KS"] = np.abs((dti["positive_cum"] / p_t) - (dti["negative_cum"] / n_t))  
    dti["IV"] = (dti["negative"] / n_t - dti["positive"] / p_t) * dti["woe"]  
  
    IV = dti["IV"].sum()  
    dti['IV'] = IV  
  
    dti = dti.reset_index()  
    dti.columns.name = None  
  
    def _cum_calc_auc(n):  
        df[new_col] = df[new_col].astype('category')  
        max_codes = df[new_col].cat.codes.max()  
  
        codes_list = sorted([n] + list(range(max_codes,n,-1)))  
        df_new = df[df[new_col].cat.codes.isin(codes_list)]  
        # df_new[var_name] = df_new[var_name].astype('category').cat.codes  
        df_new.loc[:,var_name] = df_new[var_name].astype('category').cat.codes
        unique_targets = df_new[target].unique()  
        if len(unique_targets) == 1:  
            auc = np.nan  
        else:  
            auc = metrics.roc_auc_score(df_new[target],df_new[var_name])  
            auc = auc if auc>0.5 else 1-auc  
        return auc  
    dti[new_col] = dti[new_col].astype('category')  
    dti['auc_cum'] = dti[new_col].cat.codes.map(_cum_calc_auc)  
  
    dti.rename({ new_col: 'bin'}, axis=1, inplace=True)  
    dti.insert(0, "target", [target] * dti.shape[0])  
    dti.insert(0, "var", [var_name] * dti.shape[0])  
    dti.drop(columns=["negative_cum", "positive_cum"], inplace=True)  
    return dti, bin_edges

In [4]:
def numeric_process(df, cols):  
    """  
    数值型特征处理函数，缺失值设为-999  
    :param df:  dataframe    :param cols:   数值型特征  
    :return:  dataframe  
    """    
    for col in cols:  
        df.loc[:, col] = df[col].replace('', -999)  
        df.loc[:,col].fillna(-999, inplace=True)  
        df.loc[:, col] = np.array(df[col], dtype=np.float32)  
    return df  

In [5]:
def numeric_process(df, cols):  
    """  
    数值型特征处理函数，缺失值设为-999  
    :param df:  dataframe    :param cols:   数值型特征  
    :return:  dataframe  
    """    
    for col in cols:  
        df.loc[:, col] = df[col].replace('', -999)  
        df.loc[:,col].fillna(-999, inplace=True)  
        df.loc[:, col] = np.array(df[col], dtype=np.float32)  
    return df

def time_trans(time, country_id):
    """
    时间转换
    :param time:  时间
    :param country_id: 国家id，如'mx'
    :return:  格式化后的时间
    """
    tz = GenericConfigConstant.COUNTRY_TIME_ZONE[country_id.lower()]
    time_len = len(str(time))
    time_format = "%a %b %d %H:%M:%S GMT%z %Y"
    if time_len == 10:
        format_time = datetime.utcfromtimestamp(int(time)) + timedelta(hours=tz)
    elif time_len == 13:
        format_time = datetime.utcfromtimestamp(int(time) // 1000) + timedelta(hours=tz)
    elif time_len == 34:
        format_time = datetime.strptime(str(time), time_format)
    else:
        raise ValueError('time type error: time format is not standard')

    return format_time


In [6]:
class SmsUtils:
    STOP_WORDS = {
        'mx': ['han', 'al', 'contra', 'esto', 'tengas', 'esta', 'eres', 'tú', 'nuestra', 'estuvieron', 'sois', 'con',
               'tuvo', 'estuvieses', 'esté', 'fuésemos', 'tendrán', 'sentida', 'también', 'ese', 'estad', 'nosotras',
               'tenga', 'vuestra', 'tuyo', 'e', 'nosotros', 'sería', 'porque', 'tuviese', 'estamos', 'seamos',
               'nuestro', 'sin', 'tenían', 'eran', 'poco', 'será', 'vuestros', 'hubiera', 'suya', 'desde', 'nos',
               'quien', 'su', 'tenidas', 'serás', 'estuvimos', 'les', 'estoy', 'suyos', 'estuvierais', 'sentid',
               'estuvieran', 'qué', 'habiendo', 'muchos', 'por', 'habida', 'tenida', 'fui', 'nuestras', 'mío',
               'estaban', 'tenía', 'fuese', 'sean', 'éramos', 'otro', 'algunas', 'habrán', 'tenemos', 'tuvisteis',
               'habríamos', 'hubo', 'nada', 'tendríamos', 'hubieran', 'teníais', 'son', 'esos', 'estuve', 'tenidos',
               'ellos', 'o', 'habidos', 'seríais', 'seré', 'estés', 'estaría', 'habréis', 'somos', 'esa', 'tenías',
               'teníamos', 'otros', 'hube', 'pero', 'estuvieseis', 'estada', 'tenéis', 'soy', 'tuviésemos', 'fueras',
               'estar', 'tengamos', 'este', 'mí', 'estuviese', 'míos', 'tuviera', 'una', 'entre', 'tiene', 'mi',
               'hubierais', 'habíais', 'estarás', 'ti', 'estaríamos', 'un', 'hubiéramos', 'él', 'más', 'seríamos',
               'tendría', 'le', 'tuviéramos', 'tuviesen', 'estaré', 'ante', 'sintiendo', 'las', 'mucho', 'se', 'ni',
               'hay', 'fuesen', 'habíamos', 'estuvo', 'estabas', 'es', 'suyo', 'estaremos', 'estuviste', 'tuvieseis',
               'habrías', 'estarías', 'serán', 'hubiesen', 'tuviste', 'el', 'habré', 'estuviesen', 'tus', 'tuvieron',
               'estáis', 'muy', 'estará', 'habrá', 'habías', 'seáis', 'hayáis', 'fuéramos', 'hayamos', 'todos', 'algo',
               'otras', 'habría', 'hubieses', 'hemos', 'habríais', 'para', 'otra', 'tanto', 'en', 'había', 'habrían',
               'quienes', 'a', 'estadas', 'los', 'tendrías', 'como', 'te', 'estuvisteis', 'tendrás', 'hubieron', 'haya',
               'sobre', 'estén', 'durante', 'serías', 'está', 'vuestro', 'ella', 'teniendo', 'fueseis', 'todo', 'yo',
               'tendremos', 'estéis', 'hubiste', 'estado', 'hubieras', 'tenido', 'del', 'tuyos', 'estuviésemos',
               'habido', 'tuvieran', 'era', 'tuyas', 'de', 'hayas', 'la', 'están', 'mis', 'estarán', 'estaréis',
               'habidas', 'suyas', 'tengáis', 'y', 'vuestras', 'vosotras', 'uno', 'estaba', 'erais', 'fuimos', 'siente',
               'serían', 'no', 'hubisteis', 'fue', 'ya', 'esas', 'estaríais', 'tendréis', 'cual', 'hubieseis', 'habéis',
               'antes', 'eras', 'estados', 'tienen', 'tuya', 'estas', 'ellas', 'seas', 'seremos', 'tuve', 'sentidas',
               'eso', 'tendrían', 'algunos', 'sentidos', 'estás', 'hayan', 'tengo', 'sentido', 'fueran', 'hubiese',
               'fuerais', 'estuviéramos', 'fuera', 'tened', 'donde', 'lo', 'tienes', 'hubiésemos', 'sea', 'estábamos',
               'habrás', 'estarían', 'he', 'fuisteis', 'nuestros', 'has', 'tendríais', 'tendrá', 'fueron', 'hubimos',
               'habían', 'sí', 'ha', 'tuvieras', 'vosotros', 'tuvierais', 'estemos', 'tu', 'os', 'mías', 'que',
               'estuvieras', 'estando', 'seréis', 'estuviera', 'sus', 'fuiste', 'fueses', 'tendré', 'tuvimos',
               'estabais', 'mía', 'tuvieses', 'unos', 'habremos', 'estos', 'tengan', 'me', 'hasta', 'cuando']

    }

class GenericConfigConstant:
    """
    generic config file
    """
    EPS = 1e-30
    COUNTRY_ID = ["mx", "cl", "pe", "co", "ec", "th", "ph", "in", "id", "ng", "tz"]
    COUNTRY_CODE = {"mx": "+52",
                    "cl": "+56",
                    "pe": "+51",
                    "co": "+57",
                    "ec": "+593",
                    "th": "+66",
                    "ph": "+63",
                    "in": "+91",
                    "id": "+62",
                    "ng": "+234",
                    "tz": "+255"}
    COUNTRY_PHONE_LEN = {"mx": 10,
                         "cl": 10,
                         "pe": 10,
                         "co": 10,
                         "ec": 10,
                         "th": 10,
                         "ph": 10,
                         "in": 10,
                         "id": 10,
                         "ng": 10,
                         "tz": 10}

    COUNTRY_TIME_ZONE = {"mx": -6,
                         "cl": -4,
                         "pe": -5,
                         "co": -5,
                         "ec": -5,
                         "th": 7,
                         "ph": 8,
                         "in": 5.5,
                         "id": 7,
                         "ng": 1,
                         "tz": 3}


In [8]:
def process_sms_data(user_sms: pd.DataFrame,
                     apply_time: pd.Timestamp,
                     country_id: str):
    """
    处理传入的sms数据，list of json
    Args:
        user_sms:用户短信数据
        apply_time:申请时间
        country_id:国家id
    """

    assert country_id in GenericConfigConstant.COUNTRY_ID, "country id not in list, Please input correct country id"
    if user_sms is None or user_sms.shape[0] == 0:
        return pd.DataFrame(columns=['body', 'phone', 'read', 'src_phone', 'time', 'type', 'time_day', 'hour', 'weekday', 'month', 'words', 'sender'])
    
    user_sms = user_sms[user_sms['time'].apply(lambda x: len(str(x)) in [10, 13, 34])]
    user_sms.loc[:,'time'] = user_sms['time'].apply(lambda x: time_trans(x, country_id))
    user_sms.loc[:,'time'] = pd.to_datetime(user_sms['time'], unit='ms') + pd.Timedelta(hours=GenericConfigConstant.COUNTRY_TIME_ZONE[country_id])
    user_sms = user_sms[user_sms['time'] < apply_time]

    # 每个人选取最近的3000条短信
    user_sms = user_sms.sort_values(by='time', ascending=False).head(3000)

    user_sms['time_day'] = user_sms['time'].dt.date
    # sms_data['apply_day'] = apply_time.date()
    user_sms['hour'] = user_sms['time'].dt.hour
    user_sms['weekday'] = user_sms['time'].dt.weekday
    user_sms['month'] = user_sms['time'].dt.month
    # sms_data['week'] = sms_data['time'].dt.week

    # 对'body'字段进行预处理
    user_sms['body'] = user_sms['body'].apply(lambda x: x.lower())
    user_sms['words'] = user_sms['body'].apply(lambda x: set(re.compile(r'\b[Ññáéíóúü¡A-Za-z]+\b').findall(x)))
    user_sms['words'] = user_sms['words'].apply(
        lambda x: [i for i in x if i not in SmsUtils.STOP_WORDS[country_id]])
    user_sms['words'] = user_sms['words'].apply(lambda x: [i for i in x if len(i) > 2])
    user_sms['words'] = user_sms['words'].apply(lambda x: list(map(str, x)))
    user_sms['sender'] = user_sms['phone'].apply(lambda x: normalize_phone(x, country_id))
    return user_sms

def process_app_data(app_data, apply_time,country_id):
    """
    处理传入的app数据，list of json
    Args:
        app_data:app数据
        apply_time:申请时间
        country_id:国家id
    """

    assert country_id in GenericConfigConstant.COUNTRY_ID, "country id not in list, Please input correct country id"
    if app_data is None or app_data.shape[0] == 0:
        return pd.DataFrame(columns=['app_name', 'fi_time', 'lu_time', 'fi_day', 'fi_hour', 'fi_weekday', 'fi_month',
                                     'lu_day', 'lu_hour', 'lu_weekday', 'lu_month'])

    # app_data = json.loads(app_data)
    # app_df = pd.DataFrame(app_data)
    # 将时间戳转成country_id对应的时间
    app_data['fi_time'] = pd.to_datetime(app_data['fi_time'], unit='ms') + pd.Timedelta(hours=GenericConfigConstant.COUNTRY_TIME_ZONE[country_id])
    app_data['lu_time'] = pd.to_datetime(app_data['lu_time'], unit='ms') + pd.Timedelta(hours=GenericConfigConstant.COUNTRY_TIME_ZONE[country_id])
    app_data = app_data[app_data['fi_time'] < apply_time]

    # 每个人选取最近的3000条短信
    app_data = app_data.sort_values(by='fi_time', ascending=False).head(3000)

    # 时间相关处理
    app_data['fi_day'] = app_data['fi_time'].dt.date
    app_data['fi_hour'] = app_data['fi_time'].dt.hour
    app_data['fi_weekday'] = app_data['fi_time'].dt.weekday
    app_data['fi_month'] = app_data['fi_time'].dt.month

    app_data['lu_day'] = app_data['lu_time'].dt.date
    app_data['lu_hour'] = app_data['lu_time'].dt.hour
    app_data['lu_weekday'] = app_data['lu_time'].dt.weekday
    app_data['lu_month'] = app_data['lu_time'].dt.month

    app_data['app_name'] = app_data['app_name'].apply(lambda x: x.lower())
    return app_data

def process_call_data(call_data, apply_time,country_id):

    # TODO call_data 不同国家不一样
    """
    处理传入的call数据，list of json
    Args:
        call_data:call数据
        apply_time:申请时间
        country_id:国家id
    """

    # TODO apply_time的类型
    # assert len(call_data) > 0, "call data is empty"


    # call_data = json.loads(call_data)
    # call_df = pd.DataFrame(call_data)
    # 将时间戳转成country_id对应的时间
    call_data['time'] = pd.to_datetime(call_data['time'], unit='ms') + pd.Timedelta(hours=GenericConfigConstant.COUNTRY_TIME_ZONE[country_id])
    call_data = call_data[call_data['time'] < apply_time]

    # 每个人选取最近的3000条短信
    call_data = call_data.sort_values(by='time', ascending=False).head(3000)

    if call_data.shape[0] != 0:
        # 时间相关处理
        call_data['time_day'] = call_data['time'].dt.date
        call_data['hour'] = call_data['time'].dt.hour
        call_data['weekday'] = call_data['time'].dt.weekday
        call_data['month'] = call_data['time'].dt.month
        # call_data['week'] = call_data['time'].dt.week

        call_data['src_phone'] = call_data['src_phone'].apply(lambda x: normalize_phone(x))
        call_data['dst_phone'] = call_data['dst_phone'].apply(lambda x: normalize_phone(x))
        return call_data
    else:
        return None

In [4]:
def generate_app_config_data(
        apply_time: str,
        rlevel_bins: int,
        num_configs: int
):
    """
    生成app配置数据，前三个月的数据
    :param apply_time: 申请时间
    :param rlevel_bins: rlevel分箱数
    :param num_configs: 配置文件（月份）个数\滑动窗口大小
    """
    now_month = datetime.strptime(apply_time[:7], '%Y-%m')
    config_all = pd.DataFrame()
    num_of_configs = 0
    # 遍历当前文件夹下的所有文件
    for file in os.listdir(os.getcwd()):
        if file.startswith('app_overdue_rate'):
            config_month = datetime.strptime(file.split('_')[-1].split('.')[0], '%Y-%m')
            month_diff = (now_month.year - config_month.year) * 12 + now_month.month - config_month.month
            if 1 < month_diff <= num_configs+1:
                config = pd.read_parquet(file)
                mask = (config['total_count'] > 50)     # 滑动窗口中的每个月的app的总数大于50
                config = config[mask]
                config['overdue_rate'] = config['bad_count'] / config['total_count']
                config_all = pd.concat([config_all, config], axis=0)
                num_of_configs += 1

    if num_of_configs > 1:
        config_all = config_all[config_all.groupby('app')['total_count'].transform('count') == num_of_configs]  # 选取在每个月都出现的app
        config_all = config_all[config_all.groupby('app')['overdue_rate'].transform('std') < config_all.groupby('app')['overdue_rate'].transform('std').quantile(0.9)] # 去除overdue_rate的标准差大于0.9分位数的app
        config_all = config_all.groupby('app').agg({'bad_count': 'sum', 'total_count': 'sum'}).reset_index()
        config_all['overdue_rate'] = config_all['bad_count'] / config_all['total_count']
        config_all[:,'overdue_bin_freq'] = pd.qcut(config_all['overdue_rate'], rlevel_bins, labels=False, duplicates='drop')
        config_all[:, 'level_freq'] = config_all['overdue_rate'].cat.codes+1
        config_all[:, 'overdue_bin_dist'] = pd.cut(config_all['overdue_rate'], rlevel_bins, labels=False, duplicates='drop')
        config_all[:, 'level_dist'] = config_all['overdue_rate'].cat.codes+1
        print('freq', config_all['level_freq'].value_counts())
        print('dist', config_all['level_dist'].value_counts())
    return config_all


def generate_sender_config_data(apply_time:str, rlevel_bins:int, num_configs:int, country_id='mx'):
    """
    生成配置数据，前三个月的数据
    :param apply_time: 
    :param rlevel_bins: 分箱数
    :param num_configs: 配置文件（月份）个数\滑动窗口大小
    :param country_id: 
    :return: 
    """
    now_month = datetime.strptime(apply_time[:7], '%Y-%m')
    config_all = pd.DataFrame()
    num_of_configs = 0  # 记录加在了多少个配置文件
    for file in os.listdir(os.getcwd()):
        if file.startswith('sender_overdue_rate'):
            config_month = datetime.strptime(file.split('_')[-1].split('.')[0], '%Y-%m')
            month_diff = (now_month.year - config_month.year) * 12 + now_month.month - config_month.month
            if 1 < month_diff <= num_configs+1:
                config = pd.read_parquet(file)
                mask = (config['total_count'] > 50) 
                config = config[mask]
                config['overdue_rate'] = config['bad_count'] / config['total_count']
                config_all = pd.concat([config_all, config], axis=0)
                num_of_configs += 1
                
                
    if num_of_configs > 1:
        config_all = config_all[config_all.groupby('sender')['total_count'].transform('count') == num_of_configs]
        config_all = config_all[config_all.groupby('sender')['overdue_rate'].transform('std') < config_all.groupby('sender')['overdue_rate'].transform('std').quantile(0.9)]
        config_all = config_all.groupby('sender').agg({'bad_count': 'sum', 'total_count': 'sum'}).reset_index()
        config_all['overdue_rate'] = config_all['bad_count'] / config_all['total_count']
        config_all[:,'overdue_bin_freq'] = pd.qcut(config_all['overdue_rate'], rlevel_bins, labels=False, duplicates='drop')
        config_all[:, 'level_freq'] = config_all['overdue_rate'].cat.codes+1
        config_all[:, 'overdue_bin_dist'] = pd.cut(config_all['overdue_rate'], rlevel_bins, labels=False, duplicates='drop')
        config_all[:, 'level_dist'] = config_all['overdue_rate'].cat.codes+1
        print('freq', config_all['level_freq'].value_counts())
        print('dist', config_all['level_dist'].value_counts())
    return config_all

def generate_word_config_data(apply_time:str, rlevel_bins:int, num_configs:int, country_id='mx'):
    """
    生成配置数据，前三个月的数据
    :param apply_time: 
    :param rlevel_bins:  分箱数
    :param num_configs:  配置文件（月份）个数\滑动窗口大小
    :param country_id: 
    :return: 
    """
    now_month = datetime.strptime(apply_time[:7], '%Y-%m')
    config_all = pd.DataFrame()
    num_of_configs = 0  # 记录加在了多少个配置文件
    for file in os.listdir(os.getcwd()):
        if file.startswith('word_overdue_rate'):
            config_month = datetime.strptime(file.split('_')[-1].split('.')[0], '%Y-%m')
            month_diff = (now_month.year - config_month.year) * 12 + now_month.month - config_month.month
            if 1 < month_diff <= num_configs+1:
                config = pd.read_parquet(file)
                mask = (config['total_count'] > 100) 
                config = config[mask]
                config['overdue_rate'] = config['bad_count'] / config['total_count']
                config_all = pd.concat([config_all, config], axis=0)
                num_of_configs += 1
                
                
    if num_of_configs > 1:
        config_all = config_all[config_all.groupby('word')['total_count'].transform('count') == num_of_configs]
        config_all = config_all[config_all.groupby('word')['overdue_rate'].transform('std') < config_all.groupby('word')['overdue_rate'].transform('std').quantile(0.9)]
        config_all = config_all.groupby('word').agg({'bad_count': 'sum', 'total_count': 'sum'}).reset_index()
        config_all['overdue_rate'] = config_all['bad_count'] / config_all['total_count']
        config_all[:,'overdue_bin_freq'] = pd.qcut(config_all['overdue_rate'], rlevel_bins, labels=False, duplicates='drop')
        config_all[:, 'level_freq'] = config_all['overdue_rate'].cat.codes+1
        config_all[:, 'overdue_bin_dist'] = pd.cut(config_all['overdue_rate'], rlevel_bins, labels=False, duplicates='drop')
        config_all[:, 'level_dist'] = config_all['overdue_rate'].cat.codes+1
        print('freq', config_all['level_freq'].value_counts())
        print('dist', config_all['level_dist'].value_counts())
    return config_all
    

# 特征计算

## SMS

In [12]:
class SmsConfigConstant:
    """
    短信配置常量
    """
    INTERVAL_LIST = [1, 3, 7, 15, 30, 60, 90, 180, 360, 'all']
    TIME_PERIODS = [(0, 5, 'early_morning'), (6, 10, 'morning'), (11, 13, 'noon'), (14, 17, 'afternoon'),
                    (18, 23, 'night')]
        
class SmsOverdueRateV1:
    """
    短信中word和sender的逾期率特征
    """


    @staticmethod
    def extract_sender_rlevel_cnt_features(df, time_col, config_all, apply_time):
        """
        提取 周期内 sender 数量类特征，包括数量和占比
        Args:
            df: input dataframe
            time_col: time column
            config_all: config data  包含

        Returns:
            feature dict
        """
        feature_dict = {}
        apply_time = pd.to_datetime(apply_time)
        for risk_level in config_all['risk_level'].unique():
            senders = set(config_all[config_all['risk_level'] == risk_level]['sender'])
            for time_window in SmsConfigConstant.INTERVAL_LIST:
                if df.empty:
                    feature_dict[f'sender_cnt_rlevel{risk_level}_d{time_window}'] = -999
                    feature_dict[f'sender_ratio_rlevel{risk_level}_d{time_window}'] = -999
                    feature_dict[f'sender_cnt_rlevel{risk_level}_d{time_window}_max'] = -999
                    feature_dict[f'sender_cnt_rlevel{risk_level}_d{time_window}_min'] = -999
                    feature_dict[f'sender_cnt_rlevel{risk_level}_d{time_window}_mean'] = -999
                    feature_dict[f'sender_cnt_rlevel{risk_level}_d{time_window}_std'] = -999
                    feature_dict[f'sender_cnt_rlevel{risk_level}_d{time_window}_top1'] = ''
                else:
                    if time_window == 'all':
                        time_data = df[df[time_col] <= apply_time]
                    else:
                        time_window_ = pd.Timedelta(days=time_window)
                        time_data = df[(df[time_col] >= apply_time - time_window_) & (df[time_col] <= apply_time)]
                    # 去重后的sender数量
                    sender_count_nodup = len(set(time_data['sender']) & senders)

                    # sender数量、占比
                    feature_dict[f'sender_cnt_rlevel{risk_level}_d{time_window}'] = sender_count_nodup
                    if len(time_data) == 0:
                        feature_dict[f'sender_ratio_rlevel{risk_level}_d{time_window}'] = -99
                    else:
                        feature_dict[f'sender_ratio_rlevel{risk_level}_d{time_window}'] = sender_count_nodup / len(
                            time_data)

                    filter_sender = time_data[time_data['sender'].apply(lambda x: x in senders)]
                    if filter_sender.empty:
                        feature_dict[f'sender_cnt_rlevel{risk_level}_d{time_window}_top1'] = ''
                        feature_dict[f'sender_cnt_rlevel{risk_level}_d{time_window}_max'] = 0
                        feature_dict[f'sender_cnt_rlevel{risk_level}_d{time_window}_min'] = 0
                        feature_dict[f'sender_cnt_rlevel{risk_level}_d{time_window}_mean'] = 0
                        feature_dict[f'sender_cnt_rlevel{risk_level}_d{time_window}_std'] = 0
                    else:
                        sender_count = filter_sender.groupby('sender').size()
                        sender_count = sender_count.sort_values(ascending=False)

                        # sender数量的最大值、最小值、平均值、方差
                        feature_dict[f'sender_cnt_rlevel{risk_level}_d{time_window}_max'] = sender_count.max()
                        feature_dict[f'sender_cnt_rlevel{risk_level}_d{time_window}_min'] = sender_count.min()
                        feature_dict[f'sender_cnt_rlevel{risk_level}_d{time_window}_mean'] = sender_count.mean()
                        feature_dict[f'sender_cnt_rlevel{risk_level}_d{time_window}_std'] = sender_count.std()
                        feature_dict[f'sender_cnt_rlevel{risk_level}_d{time_window}_top1'] = str(sender_count.index[0])

        return feature_dict

    @staticmethod
    def extract_sender_rlevel_time_features(df, time_col, config_all, apply_time):
        """

        提取各风险等级 sender距离apply_time的最大最小时间差
        Args:
            df: input dataframe
            time_col: time column
            config_all: config data

        Returns:
            feature dict
        """
        feature_dict = {}
        apply_time = pd.to_datetime(apply_time)
        for risk_level in config_all['risk_level'].unique():
            if df.empty:
                feature_dict[f'sender_time_diff_rlevel{risk_level}_max'] = -999
                feature_dict[f'sender_time_diff_rlevel{risk_level}_min'] = -999
                feature_dict[f'sender_time_diff_rlevel{risk_level}_mean'] = -999
                feature_dict[f'sender_time_diff_rlevel{risk_level}_std'] = -999
            else:
                senders = set(config_all[config_all['risk_level'] == risk_level]['sender'])
                feature_dict[f'sender_time_diff_rlevel{risk_level}_max'] = (
                        apply_time - df[df['sender'].apply(lambda x: x in senders)][time_col]).max().days
                feature_dict[f'sender_time_diff_rlevel{risk_level}_min'] = (
                        apply_time - df[df['sender'].apply(lambda x: x in senders)][time_col]).min().days
                feature_dict[f'sender_time_diff_rlevel{risk_level}_mean'] = (
                        apply_time - df[df['sender'].apply(lambda x: x in senders)][time_col]).mean().days
                feature_dict[f'sender_time_diff_rlevel{risk_level}_std'] = (
                        apply_time - df[df['sender'].apply(lambda x: x in senders)][time_col]).std().days
        return feature_dict

    @staticmethod
    def extract_sender_rlevel_continuous_day_features(df, time_col, config_all, apply_time):
        """
        提取各风险等级 sender连续出现天数的最大值、最小值、平均值、方差
        Args:
            df: input dataframe
            time_col: time column
            config_all: config data

        Returns:
            feature dict
        """
        feature_dict = {}
        apply_time = pd.to_datetime(apply_time)
        for risk_level in config_all['risk_level'].unique():
            senders = set(config_all[config_all['risk_level'] == risk_level]['sender'])
            for time_window in SmsConfigConstant.INTERVAL_LIST:
                if df.empty:
                    feature_dict[f'sender_continuous_rlevel{risk_level}_d{time_window}_max'] = -999
                    feature_dict[f'sender_continuous_rlevel{risk_level}_d{time_window}_min'] = -999
                    feature_dict[f'sender_continuous_rlevel{risk_level}_d{time_window}_mean'] = -999
                    feature_dict[f'sender_continuous_rlevel{risk_level}_d{time_window}_std'] = -999
                else:
                    if time_window == 'all':
                        time_data = df[df[time_col] <= apply_time]
                    else:
                        time_window_ = pd.Timedelta(days=time_window)
                        time_data = df[(df[time_col] >= apply_time - time_window_) & (df[time_col] <= apply_time)]
                    time_data = time_data.sort_values(by='time_day', ascending=True)
                    time_data.reset_index(drop=True, inplace=True)
                    filtered_data = time_data[time_data['sender'].apply(lambda x: x in senders)]
                    if filtered_data.empty:
                        feature_dict[f'sender_continuous_rlevel{risk_level}_d{time_window}_max'] = 0
                        feature_dict[f'sender_continuous_rlevel{risk_level}_d{time_window}_min'] = 0
                        feature_dict[f'sender_continuous_rlevel{risk_level}_d{time_window}_mean'] = 0
                        feature_dict[f'sender_continuous_rlevel{risk_level}_d{time_window}_std'] = 0
                    elif filtered_data['time_day'].nunique() == 1:
                        feature_dict[f'sender_continuous_rlevel{risk_level}_d{time_window}_max'] = 1
                        feature_dict[f'sender_continuous_rlevel{risk_level}_d{time_window}_min'] = 1
                        feature_dict[f'sender_continuous_rlevel{risk_level}_d{time_window}_mean'] = 1
                        feature_dict[f'sender_continuous_rlevel{risk_level}_d{time_window}_std'] = 0
                    else:
                        time_day_list = filtered_data['time_day'].unique()
                        continuous_day = 1
                        continuous_day_list = []
                        for i in range(1, len(time_day_list)):
                            if (time_day_list[i] - time_day_list[i - 1]).days == 1:
                                continuous_day += 1
                            else:
                                continuous_day_list.append(continuous_day)
                                continuous_day = 1
                        continuous_day_list.append(continuous_day)
                        feature_dict[f'sender_continuous_rlevel{risk_level}_d{time_window}_max'] = max(
                            continuous_day_list)
                        feature_dict[f'sender_continuous_rlevel{risk_level}_d{time_window}_min'] = min(
                            continuous_day_list)
                        feature_dict[f'sender_continuous_rlevel{risk_level}_d{time_window}_mean'] = np.mean(
                            continuous_day_list)
                        feature_dict[f'sender_continuous_rlevel{risk_level}_d{time_window}_std'] = np.std(
                            continuous_day_list)
        return feature_dict

    @staticmethod
    def extract_sender_rlevel_shift_diff_features(df, time_col, config_all, apply_time):
        """
        提取各时间窗口、各风险等级 sender出现频次的环比和差，即相邻时间窗口的 sender出现频次的比值和差值
        Args:
            df: input dataframe
            time_col: time column
            config_all: config data

        Returns:
            feature dict
        """
        feature_dict = {}
        apply_time = pd.to_datetime(apply_time)
        for risk_level in config_all['risk_level'].unique():
            senders = set(config_all[config_all['risk_level'] == risk_level]['sender'])
            for time_window in SmsConfigConstant.INTERVAL_LIST:
                if df.empty:
                    feature_dict[f'sender_shift_diff_rlevel{risk_level}_d{time_window}'] = -999
                    feature_dict[f'sender_shift_ratio_rlevel{risk_level}_d{time_window}'] = -999
                else:
                    if time_window == 'all':
                        continue
                    else:
                        time_window_ = pd.Timedelta(days=time_window)
                        time_data = df[(df[time_col] >= apply_time - 2 * time_window_) & (df[time_col] <= apply_time)]
                    now_window_num = len(
                        set(time_data[time_data[time_col] > apply_time - time_window_]['sender']) & senders)
                    last_window_num = len(set(
                        time_data[
                            time_data[time_col].between(apply_time - 2 * time_window_, apply_time - time_window_)][
                            'sender']) & senders)
                    feature_dict[
                        f'sender_shift_diff_rlevel{risk_level}_d{time_window}'] = now_window_num - last_window_num
                    if last_window_num == 0:
                        feature_dict[f'sender_shift_ratio_rlevel{risk_level}_d{time_window}'] = -99
                    else:
                        feature_dict[
                            f'sender_shift_ratio_rlevel{risk_level}_d{time_window}'] = now_window_num / last_window_num
        return feature_dict

    @staticmethod
    def extract_sender_rlevel_time_period_features(df, time_col, config_all, apply_time):
        """
        提取各风险等级 sender出现频次的时间段特征
        Args:
            df: input dataframe
            time_col: time column
            config_all: config data

        Returns:
            feature dict
        """
        feature_dict = {}
        apply_time = pd.to_datetime(apply_time)
        for risk_level in config_all['risk_level'].unique():
            senders = set(config_all[config_all['risk_level'] == risk_level]['sender'])
            for time_window in SmsConfigConstant.INTERVAL_LIST:
                if time_window == 'all':
                    time_data = df[df[time_col] <= apply_time]
                else:
                    time_window_ = pd.Timedelta(days=time_window)
                    time_data = df[(df[time_col] >= apply_time - time_window_) & (df[time_col] <= apply_time)]
                for time_period in SmsConfigConstant.TIME_PERIODS:
                    if df.empty:
                        feature_dict[f'sender_cnt_rlevel{risk_level}_{time_period[2]}_d{time_window}'] = -999
                        feature_dict[f'sender_ratio_rlevel{risk_level}_{time_period[2]}_d{time_window}'] = -999
                    else:
                        time_data_period = time_data[time_data['hour'].between(time_period[0], time_period[1])]
                        feature_dict[f'sender_cnt_rlevel{risk_level}_{time_period[2]}_d{time_window}'] = len(
                            set(time_data_period['sender']) & senders)
                        if len(time_data_period) == 0:
                            feature_dict[f'sender_ratio_rlevel{risk_level}_{time_period[2]}_d{time_window}'] = -99
                        else:
                            feature_dict[f'sender_ratio_rlevel{risk_level}_{time_period[2]}_d{time_window}'] = len(
                                set(time_data_period['sender']) & senders) / len(time_data_period)
        return feature_dict



    @staticmethod
    def extract_word_rlevel_cnt_features(df, time_col, config_all, apply_time):
        """
        提取 周期内 word 数量类特征，包括数量和占比
        Args:
            df: input dataframe
            time_col: time column
            config_all: config data

        Returns:
            feature dict
        """
        feature_dict = {}
        apply_time = pd.to_datetime(apply_time)
        for risk_level in config_all['risk_level'].unique():
            words = set(config_all[config_all['risk_level'] == risk_level]['word'])
            for time_window in SmsConfigConstant.INTERVAL_LIST:
                if df.empty:
                    feature_dict[f'word_cnt_rlevel{risk_level}_d{time_window}'] = -999
                    feature_dict[f'word_ratio_rlevel{risk_level}_d{time_window}'] = -999
                    feature_dict[f'word_cnt_rlevel{risk_level}_d{time_window}_max'] = -999
                    feature_dict[f'word_cnt_rlevel{risk_level}_d{time_window}_min'] = -999
                    feature_dict[f'word_cnt_rlevel{risk_level}_d{time_window}_mean'] = -999
                    feature_dict[f'word_cnt_rlevel{risk_level}_d{time_window}_std'] = -999
                else:
                    if time_window == 'all':
                        time_data = df[df[time_col] <= apply_time]
                    else:
                        time_window_ = pd.Timedelta(days=time_window)
                        time_data = df[(df[time_col] >= apply_time - time_window_) & (df[time_col] <= apply_time)]
                    # time_data 格式
                    # time_data['words']是列表
                    word_count_nodup = sum([len(words & set(x)) for x in time_data['words']])

                    feature_dict[f'word_cnt_rlevel{risk_level}_d{time_window}'] = word_count_nodup

                    if sum([len(x) for x in time_data['words']]) == 0:
                        feature_dict[f'word_ratio_rlevel{risk_level}_d{time_window}'] = -99
                    else:
                        feature_dict[f'word_ratio_rlevel{risk_level}_d{time_window}'] = word_count_nodup / sum(
                            [len(x) for x in time_data['words']])

                    filtered_data = time_data[time_data['words'].apply(lambda x: len(set(x) & words) > 0)]
                    if filtered_data.empty:
                        feature_dict[f'word_cnt_rlevel{risk_level}_d{time_window}_max'] = 0
                        feature_dict[f'word_cnt_rlevel{risk_level}_d{time_window}_min'] = 0
                        feature_dict[f'word_cnt_rlevel{risk_level}_d{time_window}_mean'] = 0
                        feature_dict[f'word_cnt_rlevel{risk_level}_d{time_window}_std'] = 0
                    else:
                        word_count = time_data[time_data['words'].apply(lambda x: len(set(x) & words) > 0)].explode(
                            'words').groupby('words').size()

                        word_count = word_count.sort_values(ascending=False)

                        feature_dict[f'word_cnt_rlevel{risk_level}_d{time_window}_max'] = word_count.max()
                        feature_dict[f'word_cnt_rlevel{risk_level}_d{time_window}_min'] = word_count.min()
                        feature_dict[f'word_cnt_rlevel{risk_level}_d{time_window}_mean'] = word_count.mean()
                        feature_dict[f'word_cnt_rlevel{risk_level}_d{time_window}_std'] = word_count.std()

        return feature_dict

    @staticmethod
    def extract_word_rlevel_time_features(df, time_col, config_all, apply_time):
        """

        提取各风险等级 word距离apply_time的最大最小时间差
        Args:
            df: input dataframe
            time_col: time column
            config_all: config data

        Returns:
            feature dict
        """
        feature_dict = {}
        apply_time = pd.to_datetime(apply_time)
        for risk_level in config_all['risk_level'].unique():
            if df.empty:
                feature_dict[f'word_time_diff_rlevel{risk_level}_max'] = -999
                feature_dict[f'word_time_diff_rlevel{risk_level}_min'] = -999
                feature_dict[f'word_time_diff_rlevel{risk_level}_mean'] = -999
                feature_dict[f'word_time_diff_rlevel{risk_level}_std'] = -999
            else:
                words = set(config_all[config_all['risk_level'] == risk_level]['word'])
                feature_dict[f'word_time_diff_rlevel{risk_level}_max'] = (
                        apply_time - df[df['words'].apply(lambda x: len(set(x) & words) > 0)][time_col]).max().days
                feature_dict[f'word_time_diff_rlevel{risk_level}_min'] = (
                        apply_time - df[df['words'].apply(lambda x: len(set(x) & words) > 0)][time_col]).min().days
                feature_dict[f'word_time_diff_rlevel{risk_level}_mean'] = (
                        apply_time - df[df['words'].apply(lambda x: len(set(x) & words) > 0)][time_col]).mean().days
                feature_dict[f'word_time_diff_rlevel{risk_level}_std'] = (
                        apply_time - df[df['words'].apply(lambda x: len(set(x) & words) > 0)][time_col]).std().days
        return feature_dict

    @staticmethod
    def extract_word_rlevel_continuous_day_features(df, time_col, config_all, apply_time):
        """
        提取各风险等级 word连续出现天数的最大值、最小值、平均值、方差
        Args:
            df: input dataframe
            time_col: time column
            config_all: config data

        Returns:
            feature dict
        """
        feature_dict = {}
        apply_time = pd.to_datetime(apply_time)
        for risk_level in config_all['risk_level'].unique():
            words = set(config_all[config_all['risk_level'] == risk_level]['word'])
            for time_window in SmsConfigConstant.INTERVAL_LIST:
                if df.empty:
                    feature_dict[f'word_continuous_rlevel{risk_level}_d{time_window}_max'] = -999
                    feature_dict[f'word_continuous_rlevel{risk_level}_d{time_window}_min'] = -999
                    feature_dict[f'word_continuous_rlevel{risk_level}_d{time_window}_mean'] = -999
                    feature_dict[f'word_continuous_rlevel{risk_level}_d{time_window}_std'] = -999
                else:
                    if time_window == 'all':
                        time_data = df[df[time_col] <= apply_time]
                    else:
                        time_window_ = pd.Timedelta(days=time_window)
                        time_data = df[(df[time_col] >= apply_time - time_window_) & (df[time_col] <= apply_time)]
                    time_data = time_data.sort_values(by='time_day', ascending=True)
                    time_data.reset_index(drop=True, inplace=True)
                    filtered_data = time_data[time_data['words'].apply(lambda x: len(set(x) & words) > 0)]
                    if filtered_data.empty:
                        feature_dict[f'word_continuous_rlevel{risk_level}_d{time_window}_max'] = 0
                        feature_dict[f'word_continuous_rlevel{risk_level}_d{time_window}_min'] = 0
                        feature_dict[f'word_continuous_rlevel{risk_level}_d{time_window}_mean'] = 0
                        feature_dict[f'word_continuous_rlevel{risk_level}_d{time_window}_std'] = 0
                    elif filtered_data['time_day'].nunique() == 1:
                        feature_dict[f'word_continuous_rlevel{risk_level}_d{time_window}_max'] = 1
                        feature_dict[f'word_continuous_rlevel{risk_level}_d{time_window}_min'] = 1
                        feature_dict[f'word_continuous_rlevel{risk_level}_d{time_window}_mean'] = 1
                        feature_dict[f'word_continuous_rlevel{risk_level}_d{time_window}_std'] = 0
                    else:
                        continuous_day = 1
                        continuous_day_list = []
                        time_data_list = filtered_data['time_day'].unique()
                        for i in range(1, len(time_data_list)):
                            if (time_data_list[i] - time_data_list[i - 1]).days == 1:
                                continuous_day += 1
                            else:
                                continuous_day_list.append(continuous_day)
                                continuous_day = 1
                        continuous_day_list.append(continuous_day)
                        feature_dict[f'word_continuous_rlevel{risk_level}_d{time_window}_max'] = max(
                            continuous_day_list)
                        feature_dict[f'word_continuous_rlevel{risk_level}_d{time_window}_min'] = min(
                            continuous_day_list)
                        feature_dict[f'word_continuous_rlevel{risk_level}_d{time_window}_mean'] = np.mean(
                            continuous_day_list)
                        feature_dict[f'word_continuous_rlevel{risk_level}_d{time_window}_std'] = np.std(
                            continuous_day_list)
        return feature_dict

    @staticmethod
    def extract_word_rlevel_shift_diff_features(df, time_col, config_all, apply_time):
        """
        提取各时间窗口、各风险等级 word出现频次的环比和差，即相邻时间窗口的 word出现频次的比值和差值
        Args:
            df: input dataframe
            time_col: time column
            config_all: config data

        Returns:
            feature dict
        """
        feature_dict = {}
        apply_time = pd.to_datetime(apply_time)
        for risk_level in config_all['risk_level'].unique():
            words = set(config_all[config_all['risk_level'] == risk_level]['word'])
            for time_window in SmsConfigConstant.INTERVAL_LIST:
                if df.empty:
                    feature_dict[f'word_shift_diff_rlevel{risk_level}_d{time_window}'] = -999
                    feature_dict[f'word_shift_ratio_rlevel{risk_level}_d{time_window}'] = -999
                else:
                    if time_window == 'all':
                        continue
                    else:
                        time_window_ = pd.Timedelta(days=time_window)
                        time_data = df[(df[time_col] >= apply_time - 2 * time_window_) & (df[time_col] <= apply_time)]

                    feature_dict[f'word_shift_diff_rlevel{risk_level}_d{time_window}'] = \
                        time_data[time_data[time_col] > apply_time - time_window_]['words'].apply(
                            lambda x: len(set(x) & words)).sum() - \
                        time_data[
                            time_data[time_col].between(apply_time - 2 * time_window_, apply_time - time_window_)][
                            'words'].apply(lambda x: len(set(x) & words)).sum()

                    if time_data[time_data[time_col].between(apply_time - 2 * time_window_, apply_time - time_window_)][
                        'words'].apply(lambda x: len(set(x) & words)).sum() == 0:
                        feature_dict[f'word_shift_ratio_rlevel{risk_level}_d{time_window}'] = -99
                    else:
                        feature_dict[f'word_shift_ratio_rlevel{risk_level}_d{time_window}'] = \
                            time_data[time_data[time_col] > apply_time - time_window_]['words'].apply(
                                lambda x: len(set(x) & words)).sum() / \
                            time_data[
                                time_data[time_col].between(apply_time - 2 * time_window_, apply_time - time_window_)][
                                'words'].apply(lambda x: len(set(x) & words)).sum()

        return feature_dict

    @staticmethod
    def extract_word_rlevel_time_period_features(df, time_col, config_all, apply_time):
        """
        提取不同time_periods、不同风险等级的word出现频次
        Args:
            df: input dataframe
            time_col: time column
            config_all: config data

        Returns:
            feature dict
        """
        feature_dict = {}
        apply_time = pd.to_datetime(apply_time)
        for risk_level in config_all['risk_level'].unique():
            words = set(config_all[config_all['risk_level'] == risk_level]['word'])
            for time_window in SmsConfigConstant.INTERVAL_LIST:
                if time_window == 'all':
                    time_data = df[df[time_col] <= apply_time]
                else:
                    time_window_ = pd.Timedelta(days=time_window)
                    time_data = df[(df[time_col] >= apply_time - time_window_) & (df[time_col] <= apply_time)]
                for time_period in SmsConfigConstant.TIME_PERIODS:
                    if df.empty:
                        feature_dict[f'word_cnt_rlevel{risk_level}_{time_period[2]}_d{time_window}'] = -999
                        feature_dict[f'word_ratio_rlevel{risk_level}_{time_period[2]}_d{time_window}'] = -999
                    else:
                        time_data_period = time_data[time_data['hour'].between(time_period[0], time_period[1])]
                        feature_dict[f'word_cnt_rlevel{risk_level}_{time_period[2]}_d{time_window}'] = sum(
                            [len(set(words_) & words) for words_ in time_data_period['words']])

                        if sum([len(words) for words in time_data_period['words']]) == 0:
                            feature_dict[f'word_ratio_rlevel{risk_level}_{time_period[2]}_d{time_window}'] = -99
                        else:
                            feature_dict[f'word_ratio_rlevel{risk_level}_{time_period[2]}_d{time_window}'] = (
                                    feature_dict[f'word_cnt_rlevel{risk_level}_{time_period[2]}_d{time_window}']
                                    / sum([len(words) for words in time_data_period['words']]))
        return feature_dict


## APP

In [13]:
class AppConfigConstant:
    """
    app配置常量
    """
    INTERVAL_LIST = [1, 3, 7, 15, 30, 60, 90, 180, 360, 'all']
    TIME_PERIODS = [(0, 5, 'early_morning'), (6, 10, 'morning'), (11, 13, 'noon'), (14, 17, 'afternoon'),
                    (18, 23, 'night')]
    # WEEK_TYPES = ['weekday', 'weekend']


class AppOverdueRateV1:
    """
    app逾期率特征
    """
    @staticmethod
    def extract_app_rlevel_cnt_features(
            df: pd.DataFrame,
            time_col: str,
            config_all: pd.DataFrame,
            apply_time: str
    ):
        """
        提取app_rlevel_cnt特征
        Args:
            df: 数据
            time_col: 时间列
            config_all: app配置数据
            apply_time: 申请时间
        Returns:
            app_rlevel_cnt特征
        """
        feature_dict = {}
        apply_time = pd.to_datetime(apply_time)
        for risk_level in config_all['risk_level'].unique():
            apps = set(config_all[config_all['risk_level'] == risk_level]['app'])
            for time_window in AppConfigConstant.INTERVAL_LIST:
                if time_window == 'all':
                    time_data = df[df[time_col] <= apply_time]
                else:
                    time_window_ = pd.Timedelta(days=time_window)
                    time_data = df[(df[time_col] >= apply_time - time_window_) & (df[time_col] <= apply_time)]
                # app_count = time_data[time_data['app_package'].isin(apps)].shape[0]
                app_count = len(set(time_data[time_data['app_name'].isin(apps)]['app_name']))
                feature_dict[f'app_cnt_rlevel{risk_level}_d{time_window}'] = app_count
                if len(time_data) == 0:
                    feature_dict[f'app_ratio_rlevel{risk_level}_d{time_window}'] = 0
                else:
                    feature_dict[f'app_ratio_rlevel{risk_level}_d{time_window}'] = app_count / len(time_data)
                # feature_dict 按key排序
                feature_dict = dict(sorted(feature_dict.items(), key=lambda x: x[0]))
        return feature_dict

    @staticmethod
    def extract_app_rlevel_time_features(
            df: pd.DataFrame,
            time_col: str,
            config_all: pd.DataFrame,
            apply_time: str
    ):
        """
        提取app_rlevel_time特征
        Args:
            df: 数据
            time_col: 时间列, fi_time / lu_time
            config_all: app配置数据
            apply_time: 申请时间
        Returns:
            app_rlevel_time特征
        """
        feature_dict = {}
        apply_time = pd.to_datetime(apply_time)
        for risk_level in config_all['risk_level'].unique():
            apps = set(config_all[config_all['risk_level'] == risk_level]['app'])
            feature_dict[f'app_time_diff_rlevel{risk_level}_max'] = (
                    apply_time - df[df['app_name'].isin(apps)][time_col]).max().days
            feature_dict[f'app_time_diff_rlevel{risk_level}_min'] = (
                    apply_time - df[df['app_name'].isin(apps)][time_col]).min().days
            feature_dict[f'app_time_diff_rlevel{risk_level}_mean'] = (
                    apply_time - df[df['app_name'].isin(apps)][time_col]).mean().days
            feature_dict[f'app_time_diff_rlevel{risk_level}_std'] = (
                    apply_time - df[df['app_name'].isin(apps)][time_col]).std().days
            feature_dict = dict(sorted(feature_dict.items(), key=lambda x: x[0]))
        return feature_dict

    @staticmethod
    def extract_app_rlevel_continuous_day_features(
            df: pd.DataFrame,
            time_col: str,
            config_all: pd.DataFrame,
            apply_time: str
    ):
        """
        提取app_rlevel_continunous_day特征
        Args:
            df: 数据
            time_col: 时间列
            config_all: app配置数据
            apply_time: 申请时间
        Returns:
            app_rlevel_continunous_day特征
        """
        feature_dict = {}
        apply_time = pd.to_datetime(apply_time)
        for risk_level in config_all['risk_level'].unique():
            apps = set(config_all[config_all['risk_level'] == risk_level]['app'])
            for time_window in AppConfigConstant.INTERVAL_LIST:
                if time_window == 'all':
                    time_data = df[df[time_col] <= apply_time]
                else:
                    time_window_ = pd.Timedelta(days=time_window)
                    time_data = df[(df[time_col] >= apply_time - time_window_) & (df[time_col] <= apply_time)]
                time_data = time_data.sort_values(by='fi_day', ascending=False)
                time_data.reset_index(drop=True, inplace=True)
                time_day_list = time_data[time_data['app_name'].isin(apps)]['fi_day'].unique()
                if len(time_day_list) == 0:
                    feature_dict[f'app_continunous_rlevel{risk_level}_d{time_window}_max'] = 0
                    feature_dict[f'app_continunous_rlevel{risk_level}_d{time_window}_min'] = 0
                    feature_dict[f'app_continunous_rlevel{risk_level}_d{time_window}_mean'] = 0
                    feature_dict[f'app_continunous_rlevel{risk_level}_d{time_window}_std'] = 0
                elif len(time_day_list) == 1:
                    feature_dict[f'app_continunous_rlevel{risk_level}_d{time_window}_max'] = 1
                    feature_dict[f'app_continunous_rlevel{risk_level}_d{time_window}_min'] = 1
                    feature_dict[f'app_continunous_rlevel{risk_level}_d{time_window}_mean'] = 1
                    feature_dict[f'app_continunous_rlevel{risk_level}_d{time_window}_std'] = 0
                else:
                    continunous_day = 1
                    continunous_day_list = []
                    for i in range(1, len(time_day_list)):
                        if (time_day_list[i - 1] - time_day_list[i]).days == 1:
                            continunous_day += 1
                        else:
                            continunous_day_list.append(continunous_day)
                            continunous_day = 1
                    continunous_day_list.append(continunous_day)
                    feature_dict[f'app_continunous_rlevel{risk_level}_d{time_window}_max'] = max(continunous_day_list)
                    feature_dict[f'app_continunous_rlevel{risk_level}_d{time_window}_min'] = min(continunous_day_list)
                    feature_dict[f'app_continunous_rlevel{risk_level}_d{time_window}_mean'] = np.mean(
                        continunous_day_list)
                    feature_dict[f'app_continunous_rlevel{risk_level}_d{time_window}_std'] = np.std(
                        continunous_day_list)
            feature_dict = dict(sorted(feature_dict.items(), key=lambda x: x[0]))
        return feature_dict

    @staticmethod
    def extract_app_rlevel_shift_diff_features(
            df: pd.DataFrame,
            time_col: str,
            config_all: pd.DataFrame,
            apply_time: str
    ):
        """
        提取app_rlevel_shift_diff特征
        Args:
            df: 数据
            time_col: 时间列
            config_all: app配置数据
            apply_time: 申请时间
        Returns:
            app_rlevel_shift_diff特征
        """
        feature_dict = {}
        apply_time = pd.to_datetime(apply_time)
        for risk_level in config_all['risk_level'].unique():
            apps = set(config_all[config_all['risk_level'] == risk_level]['app'])
            for time_window in AppConfigConstant.INTERVAL_LIST:
                if time_window == 'all':
                    continue
                else:
                    time_window_ = pd.Timedelta(days=time_window)
                    time_data = df[(df[time_col] >= apply_time - 2 * time_window_) & (df[time_col] <= apply_time)]
                now_window_num = len(set(time_data[time_data[time_col] > apply_time - time_window_]['app_name']) & apps)
                last_window_num = len(set(
                    time_data[time_data[time_col].between(apply_time - 2 * time_window_, apply_time - time_window_)][
                        'app_name']) & apps)
                feature_dict[f'app_shift_diff_rlevel{risk_level}_d{time_window}'] = now_window_num - last_window_num
                if last_window_num == 0:
                    feature_dict[f'app_shift_diff_ratio_rlevel{risk_level}_d{time_window}'] = -99
                else:
                    feature_dict[f'app_shift_diff_ratio_rlevel{risk_level}_d{time_window}'] = (now_window_num - last_window_num) / last_window_num
            feature_dict = dict(sorted(feature_dict.items(), key=lambda x: x[0]))
        return feature_dict

    @staticmethod
    def extract_app_rlevel_time_period_features(
            df: pd.DataFrame,
            time_col: str,
            config_all: pd.DataFrame,
            apply_time: str
    ):
        """
        提取app_rlevel_time_period特征
        Args:
            df: 数据
            time_col: 时间列
            config_all: app配置数据
            apply_time: 申请时间
        Returns:
            app_rlevel_time_period特征
        """
        feature_dict = {}
        apply_time = pd.to_datetime(apply_time)
        for risk_level in config_all['risk_level'].unique():
            apps = set(config_all[config_all['risk_level'] == risk_level]['app'])
            for time_window in AppConfigConstant.INTERVAL_LIST:
                if time_window == 'all':
                    time_data = df[df[time_col] <= apply_time]
                else:
                    time_window_ = pd.Timedelta(days=time_window)
                    time_data = df[(df[time_col] >= apply_time - time_window_) & (df[time_col] <= apply_time)]
                for time_period in AppConfigConstant.TIME_PERIODS:
                    time_data_period = time_data[(time_data['fi_hour'].between(time_period[0], time_period[1]))]
                    feature_dict[f'app_cnt_rlevel{risk_level}_{time_period[2]}_d{time_window}'] = len(
                        set(time_data_period['app_name']) & apps)
                    if len(time_data_period) == 0:
                        feature_dict[f'app_ratio_rlevel{risk_level}_{time_period[2]}_d{time_window}'] = -99
                    else:
                        feature_dict[f'app_ratio_rlevel{risk_level}_{time_period[2]}_d{time_window}'] = len(
                            set(time_data_period['app_name']) & apps) / len(time_data_period)

            feature_dict = dict(sorted(feature_dict.items(), key=lambda x: x[0]))
        return feature_dict



In [14]:
manager = Manager()
file_names = os.listdir('sms_url')

month_files = defaultdict(list)
for file_name in file_names:
    file_name = 'sms_url/' + file_name
    try:
        month = re.findall(r'(\d{4}-\d{2})_\w+', file_name)
        month = re.findall(r'(\d{4}-\d{2})_\w+', file_name)[0][:7]
        month_files[month].append(file_name)
    except:
        print(file_name)
month_files

sms_url/raw_sms2023-06-15
sms_url/raw_sms2023-06-01
sms_url/raw_sms2023-07-13
sms_url/raw_sms2023-08-31
sms_url/raw_sms2023-07-06
sms_url/raw_sms2023-08-17
sms_url/raw_sms2023-06-08
sms_url/raw_sms2023-08-24
sms_url/raw_sms2023-08-10
sms_url/raw_sms2023-06-02
sms_url/raw_sms2023-07-14
sms_url/raw_sms2023-09-01
sms_url/raw_sms2023-06-22
sms_url/raw_sms2023-08-25
sms_url/raw_sms2023-06-29
sms_url/raw_sms2023-06-16
sms_url/raw_sms2023-09-07
sms_url/raw_sms2023-08-03
sms_url/raw_sms2023-07-20
sms_url/raw_sms2023-06-23
sms_url/raw_sms2023-06-30
sms_url/raw_sms2023-07-27
sms_url/raw_sms2023-09-02
sms_url/raw_sms2023-08-18
sms_url/raw_sms2023-06-17
sms_url/raw_sms2023-06-09
sms_url/raw_sms2023-08-04
sms_url/raw_sms2023-07-07
sms_url/raw_sms2023-07-01
sms_url/raw_sms2023-09-08
sms_url/raw_sms2023-06-03
sms_url/raw_sms2023-08-11
sms_url/raw_sms2023-06-18
sms_url/raw_sms2023-07-15
sms_url/raw_sms2023-09-03
sms_url/raw_sms2023-06-10
sms_url/raw_sms2023-07-28
sms_url/raw_sms2023-07-21
sms_url/raw_

defaultdict(list,
            {'2023-06': ['sms_url/2023-06_AM',
              'sms_url/2023-06_YM',
              'sms_url/2023-06_M'],
             '2023-07': ['sms_url/2023-07_AM',
              'sms_url/2023-07_YM',
              'sms_url/2023-07_M'],
             '2023-08': ['sms_url/2023-08_AM',
              'sms_url/2023-08_YM',
              'sms_url/2023-08_M'],
             '2023-09': ['sms_url/2023-09_AM',
              'sms_url/2023-09_YM',
              'sms_url/2023-09_M'],
             '2023-10': ['sms_url/2023-10_AM',
              'sms_url/2023-10_YM',
              'sms_url/2023-10_M'],
             '2023-11': ['sms_url/2023-11_AM',
              'sms_url/2023-11_YM',
              'sms_url/2023-11_M'],
             '2023-12': ['sms_url/2023-12_AM',
              'sms_url/2023-12_YM',
              'sms_url/2023-12_M'],
             '2024-01': ['sms_url/2024-01_AM',
              'sms_url/2024-01_YM',
              'sms_url/2024-01_M'],
             '2024-02': ['sms_

In [15]:
def calculate_psi(expected, actual, buckets=6): # test, base
    def psi(expected_array, actual_array, buckets):
        def scale_range(input, min, max):
            input += -(np.min(input))
            input /= np.max(input) / (max - min)
            input += min
            return input
        # 按照概率值分段
        breakpoints = np.arange(0, buckets + 1) / (buckets) * 100
        breakpoints = scale_range(breakpoints, np.min(expected_array), np.max(expected_array))
        expected_percents = np.histogram(expected_array, breakpoints)[0] / len(expected_array)
        # print(expected_percents)
        actual_percents = np.histogram(actual_array, breakpoints)[0] / len(actual_array)
 
        def sub_psi(test, base): # test,base
            if base == 0:
                base = 0.0001
            if test == 0:
                test = 0.0001
 
            value = (test - base) * np.log(test / base)
            return(value)
        psi_value = np.sum(sub_psi(expected_percents[i], actual_percents[i]) for i in range(0, len(expected_percents)))
        return(psi_value)
 
    if len(expected.shape) == 1:
        psi_values = np.empty(len(expected.shape))
    else:
        psi_values = np.empty(expected.shape[0])
 
    for i in range(0, len(psi_values)):
        if len(psi_values) == 1:
            psi_values = psi(expected, actual, buckets)
        else:
            psi_values[i] = psi(expected[:,i], actual[:,i], buckets)
    return(psi_values)

In [16]:
# 并行计算
def calc_app_features_parallel(row):
    """
    并行计算app特征
    :param row:  app_features_effect的一行
    :return: 拼接特征后的row
    """
    index = row['app_order_id']
    app_data = row['applist_data']
    apply_time = pd.to_datetime(row['apply_time'])
    country_id = 'mx'
    feature_dict = {} 
    if app_data is None or app_data == '' or app_data == '[]' or app_data == 'null':
        return pd.concat([row, pd.Series(feature_dict)], axis=0).to_frame().T
    app_data = json.loads(app_data)
    user_app = pd.DataFrame(app_data)
    try:
        user_app = process_app_data(user_app, apply_time, country_id)

        if user_app.shape[0] != 0:
            feature_dict.update(AppOverdueRateV1.extract_app_rlevel_cnt_features(user_app, 'fi_time', app_config_data, apply_time))
            feature_dict.update(AppOverdueRateV1.extract_app_rlevel_time_features(user_app, 'fi_time', app_config_data, apply_time))
            feature_dict.update(AppOverdueRateV1.extract_app_rlevel_continuous_day_features(user_app, 'fi_time', app_config_data, apply_time))
            feature_dict.update(AppOverdueRateV1.extract_app_rlevel_shift_diff_features(user_app, 'fi_time', app_config_data, apply_time))
            feature_dict.update(AppOverdueRateV1.extract_app_rlevel_time_period_features(user_app, 'fi_time', app_config_data, apply_time))
        row = pd.concat([row, pd.Series(feature_dict)], axis=0).to_frame().T
    except Exception as e:
        print(e)
        print(row)
        return pd.concat([row, pd.Series(feature_dict)], axis=0).to_frame().T
    return row

In [17]:
def cal_sms_features_parallel(row):
    """
    并行计算sender特征
    :param row:  sender_features_effect的一行
    :return: 拼接特征后的row
    """
    index = row['app_order_id']
    sms_data = row['sms_data']
    apply_time = pd.to_datetime(row['apply_time'])
    country_id = 'mx'
    feature_dict = {}
    if sms_data is None or sms_data == '' or sms_data == '[]' or sms_data == 'null':
        return pd.concat([row, pd.Series(feature_dict)], axis=0).to_frame().T
    sms_data = json.loads(sms_data)
    user_sms = pd.DataFrame(sms_data)
    try:
        user_sms = process_sms_data(user_sms, apply_time, country_id)
    
        if user_sms.shape[0] != 0:
            feature_dict.update(SmsOverdueRateV1.extract_word_rlevel_cnt_features(user_sms, 'time', word_config_data, apply_time))
            feature_dict.update(SmsOverdueRateV1.extract_word_rlevel_time_features(user_sms, 'time', word_config_data, apply_time))
            feature_dict.update(SmsOverdueRateV1.extract_word_rlevel_continuous_day_features(user_sms, 'time', word_config_data, apply_time))
            feature_dict.update(SmsOverdueRateV1.extract_word_rlevel_shift_diff_features(user_sms, 'time', word_config_data, apply_time))
            feature_dict.update(SmsOverdueRateV1.extract_word_rlevel_time_period_features(user_sms, 'time', word_config_data, apply_time))
            
            feature_dict.update(SmsOverdueRateV1.extract_sender_rlevel_cnt_features(user_sms, 'time', sender_config_data, apply_time))
            feature_dict.update(SmsOverdueRateV1.extract_sender_rlevel_time_features(user_sms, 'time', sender_config_data, apply_time))
            feature_dict.update(SmsOverdueRateV1.extract_sender_rlevel_continuous_day_features(user_sms, 'time', sender_config_data, apply_time))
            feature_dict.update(SmsOverdueRateV1.extract_sender_rlevel_shift_diff_features(user_sms, 'time', sender_config_data, apply_time))
            feature_dict.update(SmsOverdueRateV1.extract_sender_rlevel_time_period_features(user_sms, 'time', sender_config_data, apply_time))
        row = pd.concat([row, pd.Series(feature_dict)], axis=0).to_frame().T
    except Exception as e:
        print(e)
        print(row)
        return pd.concat([row, pd.Series(feature_dict)], axis=0).to_frame().T
    return row

# 选取5、4、3、2月的数据

In [18]:
months = ['2024-07']

In [19]:
month_files = {month: month_files[month] for month in months}
month_files

{'2024-07': ['sms_url/2024-07_YM', 'sms_url/2024-07_AM']}

In [20]:
import gc

In [ ]:
month_files = {month: month_files[month] for month in months}
for month in months:
    print("=="*10, month, "=="*10)
    for file in month_files[month]:
        if 'AM' in file:
            sms_data = pd.read_parquet(file)
    sms_data = sms_data[sms_data['agr_pd7'] == 1]
    print(sms_data['def_pd7'].value_counts())
    app_config_data = generate_app_config_data(month, 6)
    print('app_config_data:', app_config_data.shape)
    print(app_config_data.head(1))
    sender_config_data = generate_sms_config_data(month, 6, 'sender')
    print('sender_config_data:', sender_config_data.shape)
    print(sender_config_data.head(1))
    word_config_data = generate_sms_config_data(month, 6, 'word')
    print('word_config_data:', word_config_data.shape)
    print(word_config_data.head(1))
    
    with Pool(15) as pool:
        results = pool.map(cal_sms_features_parallel, [row for _, row in sms_data[['app_order_id', 'def_pd7','apply_time','sms_data']].iterrows()])
    word_features_effect = pd.concat(results, axis=0)
    word_features_effect.to_parquet(f'word_features_effect_{month}.parquet')
    pool.close()
    print('word_features_effect:', word_features_effect.shape)
    
   

==================== 2024-07 ====================
def_pd7
1    4798
0    4390
Name: count, dtype: int64
app_config_data: (2662, 5)
  app  bad_count  total_count  overdue_rate  risk_level
0          675.0       1424.0      0.474017           5
(8784, 7)
sender_config_data: (1687, 5)
  sender  bad_count  total_count  overdue_rate  risk_level
0  10080        885         2025      0.437037           5
(82373, 7)
word_config_data: (18370, 5)
  word  bad_count  total_count  overdue_rate  risk_level
0  aaa     2231.0       9781.0      0.228095           3


/tmp/ipykernel_64363/1703579224.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_sms['time'] = user_sms['time'].apply(lambda x: time_trans(x, country_id))
/tmp/ipykernel_64363/1703579224.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_sms['time'] = pd.to_datetime(user_sms['time'], unit='ms') + pd.Timedelta(hours=GenericConfigConstant.COUNTRY_TIME_ZONE[country_id])
/tmp/ipykernel_64363/1703579224.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFr

In [ ]:
month_files = {month: month_files[month] for month in months}
for month in months:
    print("=="*10, month, "=="*10)
    for file in month_files[month]:
        if 'AM' in file:
            sms_data = pd.read_parquet(file)
    sms_data = sms_data[sms_data['agr_pd7'] == 1]
    print(sms_data['def_pd7'].value_counts())
    app_config_data = generate_app_config_data(month, 6)
    print('app_config_data:', app_config_data.shape)
    print(app_config_data.head(1))
    sender_config_data = generate_sms_config_data(month, 6, 'sender')
    print('sender_config_data:', sender_config_data.shape)
    print(sender_config_data.head(1))
    word_config_data = generate_sms_config_data(month, 6, 'word')
    print('word_config_data:', word_config_data.shape)
    print(word_config_data.head(1))
    
    with Pool(15) as pool:
        results = pool.map(calc_app_features_parallel, [row for _, row in sms_data[['app_order_id', 'def_pd7','apply_time','applist_data']].iterrows()])
    app_features_effect = pd.concat(results, axis=0)
    # print(app_features_effect.dtypes) 
    
    app_features_effect.to_parquet(f'app_features_effect_{month}.parquet')
    pool.close()
    print('app_features_effect:', app_features_effect.shape)
    with Pool(15) as pool:
        results = pool.map(cal_sms_features_parallel, [row for _, row in sms_data[['app_order_id', 'def_pd7','apply_time','sms_data']].iterrows()])
    word_features_effect = pd.concat(results, axis=0)
    word_features_effect.to_parquet(f'word_features_effect_{month}.parquet')
    pool.close()
    print('word_features_effect:', word_features_effect.shape)
    del results, app_features_effect,  sms_data
    gc.collect()

# 特征分析 
numerical_univerate 每个月使用相同的分箱

In [25]:
app_feature_cols = [col for col in app_features_effect.columns if col not in ['app_order_id', 'def_pd7', 'apply_time', 'applist_data']]


NameError: name 'app_features_effect' is not defined

In [26]:
months = ['2024-05', '2024-06', '2024-07']

In [27]:
app_features_effect_lists = [pd.read_parquet(f'app_features_effect_{month}.parquet') for month in months]

In [28]:
month_pairs = [(months[i], months[i+1]) for i in range(len(months)-1)]
month_pairs

[('2024-05', '2024-06'), ('2024-06', '2024-07')]

In [29]:
def calculate_psi(expected_counts, actual_counts):
    expected_prop = expected_counts / sum(expected_counts)
    actual_prop = actual_counts / sum(actual_counts)
    
    expected_prop = np.where(expected_prop == 0, 0.0001, expected_prop)
    actual_prop = np.where(actual_prop == 0, 0.0001, actual_prop)
    
    psi = sum((actual_prop - expected_prop) * np.log(actual_prop / expected_prop))
    return psi

# 计算psi

## 计算app特征的psi

In [23]:
months = ['2024-05', '2024-06', '2024-07']

 ## 计算sms特征的psi

In [24]:
month_pairs = [(months[i], months[i+1]) for i in range(len(months)-1)]
for month_pair in month_pairs:
    feature_analysis = pd.DataFrame()
    # 对比month_pair中的两个月字符的大小，小的为actual，大的为expected_month
    expected_month =  month_pair[1] if month_pair[0] < month_pair[1] else month_pair[0]
    actual_month =  month_pair[0] if month_pair[0] < month_pair[1] else month_pair[1]
    print('expected_month:', expected_month, 'actual_month:', actual_month)
    expected_data = pd.read_parquet(f'app_features_effect_{expected_month}.parquet')
    actual_data = pd.read_parquet(f'app_features_effect_{actual_month}.parquet')
    app_feature_cols = [col for col in expected_data.columns if col not in ['app_order_id', 'def_pd7', 'apply_time', 'applist_data']]
    for app_feature in app_feature_cols:
        feature_analysis_actual, bins_edges = numerical_univerate(actual_data, app_feature, target='def_pd7', bins=10)
        if feature_analysis_actual is None:
            continue
        feature_analysis_expected, _ = numerical_univerate(expected_data, app_feature, target='def_pd7', bins=10, bins_edges=bins_edges)
        
        feature_analysis_actual = feature_analysis_actual.rename(columns={'var': f'var_{actual_month}', 'target': f'target_{actual_month}', 'bin': f'bin_{actual_month}', 'negative': f'negative_{actual_month}', 'positive': f'positive_{actual_month}', 'total': f'total_{actual_month}', 'total_rate': f'total_rate_{actual_month}', 'positive_rate': f'positive_rate_{actual_month}', 'positive_rate_cum': f'positive_rate_cum_{actual_month}', 'woe': f'woe_{actual_month}', 'LIFT': f'LIFT_{actual_month}', 'KS': f'KS_{actual_month}', 'IV': f'IV_{actual_month}', 'auc_cum': f'auc_cum_{actual_month}'})
        
        feature_analysis_expected = feature_analysis_expected.rename(columns={'var': f'var_{expected_month}', 'target': f'target_{expected_month}', 'bin': f'bin_{expected_month}', 'negative': f'negative_{expected_month}', 'positive': f'positive_{expected_month}', 'total': f'total_{expected_month}', 'total_rate': f'total_rate_{expected_month}', 'positive_rate': f'positive_rate_{expected_month}', 'positive_rate_cum': f'positive_rate_cum_{expected_month}', 'woe': f'woe_{expected_month}', 'LIFT': f'LIFT_{expected_month}', 'KS': f'KS_{expected_month}', 'IV': f'IV_{expected_month}', 'auc_cum': f'auc_cum_{expected_month}'})
        feature_analysis_ = pd.concat([feature_analysis_actual, feature_analysis_expected], axis=1)
        if feature_analysis_expected.shape[0] != feature_analysis_actual.shape[0]:
            feature_analysis = pd.concat([feature_analysis, feature_analysis_], axis=0)
            continue
        expected_rate = np.where(feature_analysis_expected[f'positive_rate_{expected_month}'] == 0, 0.0001, feature_analysis_expected[f'positive_rate_{expected_month}'])
        actual_rate = np.where(feature_analysis_actual[f'positive_rate_{actual_month}'] == 0, 0.0001, feature_analysis_actual[f'positive_rate_{actual_month}'])
        feature_analysis_['psi'] = sum((actual_rate - expected_rate) * np.log(actual_rate / expected_rate))
        feature_analysis = pd.concat([feature_analysis, feature_analysis_], axis=0)
    feature_analysis.to_excel(f'app_features_effect_{expected_month}_{actual_month}.xlsx')

expected_month: 2024-06 actual_month: 2024-05
expected_month: 2024-07 actual_month: 2024-06


In [25]:
months = ['2024-05', '2024-06', '2024-07']

In [26]:
month_pairs = [(months[i], months[i+1]) for i in range(len(months)-1)]
for month_pair in month_pairs:
    feature_analysis = pd.DataFrame()
    expected_month = month_pair[1] if month_pair[0] < month_pair[1] else month_pair[0]
    actual_month = month_pair[0] if month_pair[0] < month_pair[1] else month_pair[1]
    expected_data = pd.read_parquet(f'word_features_effect_{expected_month}.parquet')
    actual_data = pd.read_parquet(f'word_features_effect_{actual_month}.parquet')
    sms_feature_cols = [col for col in expected_data.columns if col not in ['app_order_id', 'def_pd7', 'apply_time', 'sms_data']]
    print('expected_month:', expected_month, 'actual_month:', actual_month)
    for sms_feature in sms_feature_cols:
        feature_analysis_actual, bins_edges = numerical_univerate(actual_data, sms_feature, target='def_pd7', bins=10)
        if feature_analysis_actual is None:
            continue
        feature_analysis_expected, _ = numerical_univerate(expected_data, sms_feature, target='def_pd7', bins=10, bins_edges=bins_edges)
        
        feature_analysis_actual = feature_analysis_actual.rename(columns={'var': f'var_{actual_month}', 'target': f'target_{actual_month}', 'bin': f'bin_{actual_month}', 'negative': f'negative_{actual_month}', 'positive': f'positive_{actual_month}', 'total': f'total_{actual_month}', 'total_rate': f'total_rate_{actual_month}', 'positive_rate': f'positive_rate_{actual_month}', 'positive_rate_cum': f'positive_rate_cum_{actual_month}', 'woe': f'woe_{actual_month}', 'LIFT': f'LIFT_{actual_month}', 'KS': f'KS_{actual_month}', 'IV': f'IV_{actual_month}', 'auc_cum': f'auc_cum_{actual_month}'})
        
        feature_analysis_expected = feature_analysis_expected.rename(columns={'var': f'var_{expected_month}', 'target': f'target_{expected_month}', 'bin': f'bin_{expected_month}', 'negative': f'negative_{expected_month}', 'positive': f'positive_{expected_month}', 'total': f'total_{expected_month}', 'total_rate': f'total_rate_{expected_month}', 'positive_rate': f'positive_rate_{expected_month}', 'positive_rate_cum': f'positive_rate_cum_{expected_month}', 'woe': f'woe_{expected_month}', 'LIFT': f'LIFT_{expected_month}', 'KS': f'KS_{expected_month}', 'IV': f'IV_{expected_month}', 'auc_cum': f'auc_cum_{expected_month}'})
        feature_analysis_ = pd.concat([feature_analysis_actual, feature_analysis_expected], axis=1)
        if feature_analysis_expected.shape[0] != feature_analysis_actual.shape[0]:
            feature_analysis = pd.concat([feature_analysis, feature_analysis_], axis=0)
            continue
        expected_rate = np.where(feature_analysis_expected[f'positive_rate_{expected_month}'] == 0, 0.0001, feature_analysis_expected[f'positive_rate_{expected_month}'])
        actual_rate = np.where(feature_analysis_actual[f'positive_rate_{actual_month}'] == 0, 0.0001, feature_analysis_actual[f'positive_rate_{actual_month}'])
        feature_analysis_['psi'] = sum((actual_rate - expected_rate) * np.log(actual_rate / expected_rate))
        feature_analysis = pd.concat([feature_analysis, feature_analysis_], axis=0)
    feature_analysis.to_excel(f'word_features_effect_{expected_month}_{actual_month}.xlsx')

expected_month: 2024-06 actual_month: 2024-05
expected_month: 2024-07 actual_month: 2024-06


In [136]:
# 生成app配置数据
app_config_data = generate_app_config_data('2024-05-30', 6)

In [137]:
app_config_data.shape

(2625, 5)

In [105]:
# 计算app特征，当结果和app_order_id、def_pd7保存在features_effect中
app_features_effect = sms_data[['app_order_id', 'def_pd7','apply_time','applist_data']]
app_features_effect = app_features_effect.reset_index(drop=True)
app_features_effect.shape

(6115, 4)

In [49]:
app_features_effect


,app_order_id,def_pd7,apply_time,applist_data,app_cnt_day180_rlevel0,app_cnt_day180_rlevel1,app_cnt_day180_rlevel2,app_cnt_day180_rlevel3,app_cnt_day180_rlevel4,app_cnt_day180_rlevel5,...,app_ratio_dayall_rlevel3_morning,app_ratio_dayall_rlevel3_night,app_ratio_dayall_rlevel4_afternoon,app_ratio_dayall_rlevel4_dawn,app_ratio_dayall_rlevel4_morning,app_ratio_dayall_rlevel4_night,app_ratio_dayall_rlevel5_afternoon,app_ratio_dayall_rlevel5_dawn,app_ratio_dayall_rlevel5_morning,app_ratio_dayall_rlevel5_night
0,1248158226080780288,0,2024-06-06 00:15:23,"[{""app_name"":""Billiards King"",""app_package"":""c...",4.0,1.0,1.0,7.0,4.0,34.0,...,0.127273,0.250000,0.053763,0.050505,0.051515,0.250000,0.086022,0.232323,0.048485,0.125000
1,1248160123483246592,0,2024-06-06 00:22:43,"[{""app_name"":""NeroCredit"",""app_package"":""com.c...",4.0,1.0,0.0,4.0,3.0,10.0,...,0.153846,0.000000,0.058824,0.147368,0.076923,0.142857,0.117647,0.094737,0.153846,0.428571
2,1248160707112259584,0,2024-06-06 00:25:05,"[{""app_name"":""Tethering"",""app_package"":""com.go...",0.0,3.0,1.0,5.0,7.0,5.0,...,0.142857,0.333333,0.000000,0.097938,0.250000,0.000000,0.200000,0.005155,0.035714,0.333333
3,1248161485717688320,0,2024-06-06 00:28:11,"[{""app_name"":""Tethering"",""app_package"":""com.go...",2.0,0.0,1.0,0.0,2.0,0.0,...,0.246305,0.250000,0.000000,0.125000,0.059113,0.333333,0.000000,0.125000,0.009852,0.000000
4,1248162274980843520,0,2024-06-06 00:31:35,"[{""app_name"":""Clonar teléfono"",""app_package"":""...",2.0,1.0,0.0,4.0,1.0,1.0,...,0.166667,0.272727,0.018519,0.142857,0.033333,0.181818,0.024691,0.047619,0.033333,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21214,1252499026918952960,0,2024-06-17 23:44:08,"[{""app_name"":""Tethering"",""app_package"":""com.go...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21215,1252500447567466496,0,2024-06-17 23:49:45,"[{""app_name"":""Tethering"",""app_package"":""com.go...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21216,1252501250264977408,0,2024-06-17 23:53:13,"[{""app_name"":""Radio FM"",""app_package"":""com.and...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21217,1252501578880307200,0,2024-06-17 23:54:15,"[{""app_name"":""Tethering"",""app_package"":""com.go...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [150]:
# 并行计算app_features_effect 
with Pool(15) as pool:
    results = pool.map(process_app_data_parallel, [row for _, row in app_features_effect.iterrows()])
app_features_effect = pd.DataFrame(results)

In [151]:
app_features_effect.shape

(6115, 1108)

In [152]:
app_features_effect.head()

,app_order_id,def_pd7,apply_time,applist_data,app_cnt_day15_rlevel0,app_cnt_day15_rlevel1,app_cnt_day15_rlevel2,app_cnt_day15_rlevel3,app_cnt_day15_rlevel4,app_cnt_day15_rlevel5,...,app_ratio_dayall_rlevel4_afternoon,app_ratio_dayall_rlevel4_early_morning,app_ratio_dayall_rlevel4_morning,app_ratio_dayall_rlevel4_night,app_ratio_dayall_rlevel4_noon,app_ratio_dayall_rlevel5_afternoon,app_ratio_dayall_rlevel5_early_morning,app_ratio_dayall_rlevel5_morning,app_ratio_dayall_rlevel5_night,app_ratio_dayall_rlevel5_noon
0,1243443540596805632,1,2024-05-24 00:00:42,"[{""app_name"":""Tethering"",""app_package"":""com.go...",0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.080000,0.250000,-99.000000,0.0,0.000000,0.000000,0.166667,-99.000000,0.000000
1,1243469959624314880,1,2024-05-24 01:45:41,"[{""app_name"":""Tethering"",""app_package"":""com.go...",0.0,0.0,0.0,1.0,0.0,1.0,...,0.166667,0.500000,0.093750,0.000000,0.0,0.333333,0.500000,0.031250,0.000000,0.000000
2,1243548779010248704,1,2024-05-24 06:58:53,"[{""app_name"":""Grabadora de pantalla"",""app_pack...",0.0,0.0,2.0,2.0,0.0,1.0,...,0.076923,0.166667,0.111111,0.130435,-99.0,0.153846,0.166667,0.000000,0.086957,-99.000000
3,1243549820346880000,1,2024-05-24 07:03:01,"[{""app_name"":""Tethering"",""app_package"":""com.go...",1.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.107143,0.116279,0.000000,0.0,0.000000,0.214286,0.000000,0.000000,0.000000
4,1243552282038423552,1,2024-05-24 07:12:48,"[{""app_name"":""com.android.cts.priv.ctsshim"",""a...",0.0,2.0,0.0,0.0,0.0,1.0,...,0.000000,0.157895,0.133333,0.000000,0.0,0.500000,0.078947,0.200000,0.000000,0.285714


In [153]:
print(app_features_effect.columns)

Index(['app_order_id', 'def_pd7', 'apply_time', 'applist_data',
       'app_cnt_day15_rlevel0', 'app_cnt_day15_rlevel1',
       'app_cnt_day15_rlevel2', 'app_cnt_day15_rlevel3',
       'app_cnt_day15_rlevel4', 'app_cnt_day15_rlevel5',
       ...
       'app_ratio_dayall_rlevel4_afternoon',
       'app_ratio_dayall_rlevel4_early_morning',
       'app_ratio_dayall_rlevel4_morning', 'app_ratio_dayall_rlevel4_night',
       'app_ratio_dayall_rlevel4_noon', 'app_ratio_dayall_rlevel5_afternoon',
       'app_ratio_dayall_rlevel5_early_morning',
       'app_ratio_dayall_rlevel5_morning', 'app_ratio_dayall_rlevel5_night',
       'app_ratio_dayall_rlevel5_noon'],
      dtype='object', length=1108)


In [154]:
for col in app_features_effect.columns:
    print(col)

app_order_id
def_pd7
apply_time
applist_data
app_cnt_day15_rlevel0
app_cnt_day15_rlevel1
app_cnt_day15_rlevel2
app_cnt_day15_rlevel3
app_cnt_day15_rlevel4
app_cnt_day15_rlevel5
app_cnt_day180_rlevel0
app_cnt_day180_rlevel1
app_cnt_day180_rlevel2
app_cnt_day180_rlevel3
app_cnt_day180_rlevel4
app_cnt_day180_rlevel5
app_cnt_day1_rlevel0
app_cnt_day1_rlevel1
app_cnt_day1_rlevel2
app_cnt_day1_rlevel3
app_cnt_day1_rlevel4
app_cnt_day1_rlevel5
app_cnt_day30_rlevel0
app_cnt_day30_rlevel1
app_cnt_day30_rlevel2
app_cnt_day30_rlevel3
app_cnt_day30_rlevel4
app_cnt_day30_rlevel5
app_cnt_day360_rlevel0
app_cnt_day360_rlevel1
app_cnt_day360_rlevel2
app_cnt_day360_rlevel3
app_cnt_day360_rlevel4
app_cnt_day360_rlevel5
app_cnt_day3_rlevel0
app_cnt_day3_rlevel1
app_cnt_day3_rlevel2
app_cnt_day3_rlevel3
app_cnt_day3_rlevel4
app_cnt_day3_rlevel5
app_cnt_day60_rlevel0
app_cnt_day60_rlevel1
app_cnt_day60_rlevel2
app_cnt_day60_rlevel3
app_cnt_day60_rlevel4
app_cnt_day60_rlevel5
app_cnt_day7_rlevel0
app_cnt_da

In [155]:
feature_cols = [col for col in app_features_effect.columns if col not in ['app_order_id', 'def_pd7', 'apply_time', 'applist_data']]

In [163]:
app_features_effect['app_continunous_day1_rlevel0_std'].value_counts()

app_continunous_day1_rlevel0_std
0.0    6115
Name: count, dtype: int64

In [165]:
features_effect_result = pd.DataFrame()
for col in feature_cols:
    print(col)
    result = numerical_univerate(app_features_effect, col, 'def_pd7', bins=10)
    features_effect_result = pd.concat([features_effect_result, result], axis=0)

app_cnt_day15_rlevel0
app_cnt_day15_rlevel1
app_cnt_day15_rlevel2
app_cnt_day15_rlevel3
app_cnt_day15_rlevel4
app_cnt_day15_rlevel5
app_cnt_day180_rlevel0
app_cnt_day180_rlevel1
app_cnt_day180_rlevel2
app_cnt_day180_rlevel3
app_cnt_day180_rlevel4
app_cnt_day180_rlevel5
app_cnt_day1_rlevel0
app_cnt_day1_rlevel1
app_cnt_day1_rlevel2
app_cnt_day1_rlevel3
app_cnt_day1_rlevel4
app_cnt_day1_rlevel5
app_cnt_day30_rlevel0
app_cnt_day30_rlevel1
app_cnt_day30_rlevel2
app_cnt_day30_rlevel3
app_cnt_day30_rlevel4
app_cnt_day30_rlevel5
app_cnt_day360_rlevel0
app_cnt_day360_rlevel1
app_cnt_day360_rlevel2
app_cnt_day360_rlevel3
app_cnt_day360_rlevel4
app_cnt_day360_rlevel5
app_cnt_day3_rlevel0
app_cnt_day3_rlevel1
app_cnt_day3_rlevel2
app_cnt_day3_rlevel3
app_cnt_day3_rlevel4
app_cnt_day3_rlevel5
app_cnt_day60_rlevel0
app_cnt_day60_rlevel1
app_cnt_day60_rlevel2
app_cnt_day60_rlevel3
app_cnt_day60_rlevel4
app_cnt_day60_rlevel5
app_cnt_day7_rlevel0
app_cnt_day7_rlevel1
app_cnt_day7_rlevel2
app_cnt_day7_

In [166]:
features_effect_result.to_excel('app_features_effect.xlsx')

In [167]:
features_effect_result

,var,target,bin,negative,positive,total,total_rate,positive_rate,positive_rate_cum,woe,LIFT,KS,IV,auc_cum
0,app_cnt_day15_rlevel0,def_pd7,"(1.0, 19.0]",224,198,422,0.069011,0.469194,0.469194,0.157400,0.921659,0.011013,0.001864,0.552715
1,app_cnt_day15_rlevel0,def_pd7,"(-0.001, 1.0]",2778,2915,5693,0.930989,0.512032,0.509076,-0.011818,1.005807,0.000000,0.001864,0.511912
0,app_cnt_day15_rlevel1,def_pd7,"(1.0, 13.0]",151,154,305,0.049877,0.504918,0.504918,0.016308,0.991832,0.000830,0.000014,0.582609
1,app_cnt_day15_rlevel1,def_pd7,"(-0.001, 1.0]",2851,2959,5810,0.950123,0.509294,0.509076,-0.000872,1.000429,0.000000,0.000014,0.525073
0,app_cnt_day15_rlevel2,def_pd7,"(1.0, 13.0]",158,187,345,0.056419,0.542029,0.542029,-0.129894,1.064731,0.007439,0.001025,0.572193
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,app_ratio_dayall_rlevel5_noon,def_pd7,"(0.158, 0.25]",308,396,704,0.115127,0.562500,0.542079,-0.213137,1.104943,0.026174,0.008666,0.545341
2,app_ratio_dayall_rlevel5_noon,def_pd7,"(0.1, 0.158]",270,308,578,0.094522,0.532872,0.539106,-0.094369,1.046743,0.035174,0.008666,0.515668
3,app_ratio_dayall_rlevel5_noon,def_pd7,"(0.05, 0.1]",325,306,631,0.103189,0.484945,0.524990,0.095621,0.952597,0.025210,0.008666,0.512460
4,app_ratio_dayall_rlevel5_noon,def_pd7,"(0.0, 0.05]",155,139,294,0.048078,0.472789,0.519337,0.142293,0.928720,0.018229,0.008666,0.520400


# 计算sms特征

In [20]:
sms_sender_config_data = generate_sms_config_data('2024-05-30', 6, 'sender')

2024-01-01 00:00:00 4
sender_overdue_rate_2024-01.parquet
           sender  bad_count  total_count      mean       woe
1169241     45578       7413        19062  0.388889  1.113029
1184518    telcel       5871        15359  0.382251  1.094032
1169066     43885       5216        13483  0.386858  1.107215
1184591  unotvcom       5124        13439  0.381278  1.091247
1168327     27272       5013        13113  0.382292  1.094150
2024-03-01 00:00:00 2
sender_overdue_rate_2024-03.parquet
           sender  bad_count  total_count      mean       woe
1082277     45578       5827        16193  0.359847  2.079735
1095274    telcel       4640        13144  0.353013  2.040238
1095356  unotvcom       3831        11070  0.346070  2.000115
1081466     26767       3658        10762  0.339900  1.964451
1089394     59850       3759        10437  0.360161  2.081551
2024-06-01 00:00:00 -1
2024-05-01 00:00:00 0
2024-04-01 00:00:00 1
2023-06-01 00:00:00 11
2023-07-01 00:00:00 10
2023-08-01 00:00:00 9
2023-

In [22]:
sender_features_effect = sms_data[['app_order_id', 'def_pd7','apply_time','sms_data']]
sender_features_effect = sender_features_effect.reset_index(drop=True)
sender_features_effect.shape

(6115, 4)

In [ ]:
sender_features_effect 

In [74]:
def process_sender_data_parallel(row):
    """
    并行计算sender特征
    :param row:  sender_features_effect的一行
    :return: 拼接特征后的row
    """
    index = row['app_order_id']
    sms_data = row['sms_data']
    apply_time = pd.to_datetime(row['apply_time'])
    country_id = 'MX'
    feature_dict = {}
    if sms_data == '' or sms_data == '[]' or sms_data == 'null':
        return pd.concat([row, pd.Series(feature_dict)], axis=0).to_frame().T
    sms_data = json.loads(sms_data)
    user_sms = pd.DataFrame(sms_data)
    user_sms = process_sms_data(user_sms, apply_time, country_id)
    
    
    if user_sms.shape[0] != 0:
        feature_dict.update(extract_sender_rlevel_cnt_features(user_sms, 'time', sms_sender_config_data, apply_time))
        feature_dict.update(extract_sender_rlevel_time_features(user_sms, 'time', sms_sender_config_data, apply_time))
        feature_dict.update(extract_sender_rlevel_continuous_day_features(user_sms, 'time', sms_sender_config_data, apply_time))
        feature_dict.update(extract_sender_rlevel_shift_diff_features(user_sms, 'time', sms_sender_config_data, apply_time))
        feature_dict.update(extract_sender_rlevel_time_period_features(user_sms, 'time', sms_sender_config_data, apply_time))
    row = pd.concat([row, pd.Series(feature_dict)], axis=0).to_frame().T
    return row

In [31]:
sender_features_effect.head()

,app_order_id,def_pd7,apply_time,sms_data
0,1243443540596805632,1,2024-05-24 00:00:42,"[{""body"":""ÄCreditoVelozÑ 6626 es su codigo de ..."
1,1243469959624314880,1,2024-05-24 01:45:41,"[{""body"":""<CreditoVeloz> 1426 es su codigo de ..."
2,1243548779010248704,1,2024-05-24 06:58:53,"[{""body"":""ÄCreditoVelozÑ 7477 es su codigo de ..."
3,1243549820346880000,1,2024-05-24 07:03:01,"[{""body"":""Tenemos noticias sobre tu cambio de ..."
4,1243552282038423552,1,2024-05-24 07:12:48,"[{""body"":""<CreditoVeloz> 5449 es su codigo de ..."


In [33]:
print(pd.to_datetime('2024-05-24 00:00:42'))

2024-05-24 00:00:42


In [81]:
process_sender_data_parallel(sender_features_effect.iloc[0])

Index(['body', 'phone', 'read', 'src_phone', 'time', 'type', 'time_day',
       'hour', 'weekday', 'month', 'words', 'sender'],
      dtype='object')
Index(['body', 'phone', 'read', 'src_phone', 'time', 'type', 'time_day',
       'hour', 'weekday', 'month', 'words', 'sender'],
      dtype='object')
Index(['body', 'phone', 'read', 'src_phone', 'time', 'type', 'time_day',
       'hour', 'weekday', 'month', 'words', 'sender'],
      dtype='object')
Index(['body', 'phone', 'read', 'src_phone', 'time', 'type', 'time_day',
       'hour', 'weekday', 'month', 'words', 'sender'],
      dtype='object')
Index(['body', 'phone', 'read', 'src_phone', 'time', 'type', 'time_day',
       'hour', 'weekday', 'month', 'words', 'sender'],
      dtype='object')
Index(['body', 'phone', 'read', 'src_phone', 'time', 'type', 'time_day',
       'hour', 'weekday', 'month', 'words', 'sender'],
      dtype='object')
Index(['body', 'phone', 'read', 'src_phone', 'time', 'type', 'time_day',
       'hour', 'weekday', '

,app_order_id,def_pd7,apply_time,sms_data,sender_cnt_day1_rlevel5,sender_ratio_day1_rlevel5,sender_cnt_day1_rlevel5_max,sender_cnt_day1_rlevel5_min,sender_cnt_day1_rlevel5_mean,sender_cnt_day1_rlevel5_std,...,sender_cnt_dayall_rlevel0_early_morning,sender_ratio_dayall_rlevel0_early_morning,sender_cnt_dayall_rlevel0_morning,sender_ratio_dayall_rlevel0_morning,sender_cnt_dayall_rlevel0_noon,sender_ratio_dayall_rlevel0_noon,sender_cnt_dayall_rlevel0_afternoon,sender_ratio_dayall_rlevel0_afternoon,sender_cnt_dayall_rlevel0_night,sender_ratio_dayall_rlevel0_night
0,1243443540596805632,1,2024-05-24 00:00:42,"[{""body"":""ÄCreditoVelozÑ 6626 es su codigo de ...",0,0.0,NaN,NaN,NaN,NaN,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0


In [87]:
# 并行计算sender_features_effect
with Pool(15) as pool:
    results = pool.map(process_sender_data_parallel, [row for _, row in sender_features_effect.iterrows()])

In [88]:
sender_features_effect = pd.concat(results, axis=0)

In [89]:
sender_features_effect.head()

,app_order_id,def_pd7,apply_time,sms_data,sender_cnt_day1_rlevel5,sender_ratio_day1_rlevel5,sender_cnt_day1_rlevel5_max,sender_cnt_day1_rlevel5_min,sender_cnt_day1_rlevel5_mean,sender_cnt_day1_rlevel5_std,...,sender_cnt_dayall_rlevel0_early_morning,sender_ratio_dayall_rlevel0_early_morning,sender_cnt_dayall_rlevel0_morning,sender_ratio_dayall_rlevel0_morning,sender_cnt_dayall_rlevel0_noon,sender_ratio_dayall_rlevel0_noon,sender_cnt_dayall_rlevel0_afternoon,sender_ratio_dayall_rlevel0_afternoon,sender_cnt_dayall_rlevel0_night,sender_ratio_dayall_rlevel0_night
0,1243443540596805632,1,2024-05-24 00:00:42,"[{""body"":""ÄCreditoVelozÑ 6626 es su codigo de ...",0,0.0,NaN,NaN,NaN,NaN,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
0,1243469959624314880,1,2024-05-24 01:45:41,"[{""body"":""<CreditoVeloz> 1426 es su codigo de ...",0,0.0,NaN,NaN,NaN,NaN,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
0,1243548779010248704,1,2024-05-24 06:58:53,"[{""body"":""ÄCreditoVelozÑ 7477 es su codigo de ...",1,0.5,1,1,1.0,NaN,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
0,1243549820346880000,1,2024-05-24 07:03:01,"[{""body"":""Tenemos noticias sobre tu cambio de ...",1,0.111111,1,1,1.0,NaN,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
0,1243552282038423552,1,2024-05-24 07:12:48,"[{""body"":""<CreditoVeloz> 5449 es su codigo de ...",0,0.0,NaN,NaN,NaN,NaN,...,0,0.0,0,0.0,0,0.0,0,0.0,1,0.125


In [90]:
sender_features_effect.shape

(6115, 1408)

# 特征命名和含义

In [27]:
# app特征
app_features  = pd.read_parquet('app_features_effect_2024-06.parquet').head(1)

In [28]:
app_features

,app_order_id,def_pd7,apply_time,applist_data,app_cnt_rlevel0_d1,app_cnt_rlevel0_d15,app_cnt_rlevel0_d180,app_cnt_rlevel0_d3,app_cnt_rlevel0_d30,app_cnt_rlevel0_d360,...,app_ratio_rlevel5_noon_d1,app_ratio_rlevel5_noon_d15,app_ratio_rlevel5_noon_d180,app_ratio_rlevel5_noon_d3,app_ratio_rlevel5_noon_d30,app_ratio_rlevel5_noon_d360,app_ratio_rlevel5_noon_d60,app_ratio_rlevel5_noon_d7,app_ratio_rlevel5_noon_d90,app_ratio_rlevel5_noon_dall
0,1256176301434654720,1,2024-06-28 03:16:09,"[{""app_name"":""Grabadora de pantalla"",""app_pack...",0.0,0.0,2.0,0.0,1.0,2.0,...,-99.0,0.0,0.0,-99.0,0.0,0.0,0.0,-99.0,0.0,0.0


In [29]:
app_features = app_features.columns.tolist()
len(app_features)

1096

In [30]:
app_features[1]

'def_pd7'

In [31]:
app_features_df = pd.DataFrame(app_features, columns=['feature'])
app_features_df['features_meaning'] = ''

In [32]:
app_features_df.head(1)

,feature,features_meaning
0,app_order_id,


In [35]:
# app_cnt_rlevel{risk_level}_d{time_window} 逾期率分箱为risk_level的app在time_window天内的数量
# app_ratio_rlevel{risk_level}_d{time_window} 逾期率分箱为risk_level的app在time_window天内的占比
# app_time_diff_rlevel{risk_level}_max 逾期率分箱为risk_level的app在time_window天内距apply_time的最大时间差
# app_time_diff_rlevel{risk_level}_min 逾期率分箱为risk_level的app在time_window天内距apply_time的最小时间差
# app_time_diff_rlevel{risk_level}_mean 逾期率分箱为risk_level的app在time_window天内距apply_time的平均时间差
# app_time_diff_rlevel{risk_level}_std 逾期率分箱为risk_level的app在time_window天内距apply_time的时间差的标准差
# app_continunous_rlevel{risk_level}_d{time_window}_max 逾期率分箱为risk_level的app在time_window天内连续出现的最大天数
# app_continunous_rlevel{risk_level}_d{time_window}_min 逾期率分箱为risk_level的app在time_window天内连续出现的最小天数
# app_continunous_rlevel{risk_level}_d{time_window}_mean 逾期率分箱为risk_level的app在time_window天内连续出现的平均天数
# app_continunous_rlevel{risk_level}_d{time_window}_std 逾期率分箱为risk_level的app在time_window天内连续出现的天数的标准差
# app_shift_diff_rlevel{risk_level}_d{time_window} 逾期率分箱为risk_level的app在time_window天内的变化量
# app_shift_diff_ratio_rlevel{risk_level}_d{time_window} 逾期率分箱为risk_level的app在time_window天内的变化比例
# app_cnt_rlevel{risk_level}_{time_period}_d{time_window} 逾期率分箱为risk_level的app在time_window天内在time_period时间段内的数量
# app_ratio_rlevel{risk_level}_{time_period}_d{time_window} 逾期率分箱为risk_level的app在time_window天内在time_period时间段内的占比

app_features_df = pd.DataFrame(columns=['feature', 'features_meaning'])
app_features_df['feature'] = ['app_cnt_rlevel{risk_level}_d{time_window}','app_ratio_rlevel{risk_level}_d{time_window}', 'app_time_diff_rlevel{risk_level}_max', 'app_time_diff_rlevel{risk_level}_min', 'app_time_diff_rlevel{risk_level}_mean', 'app_time_diff_rlevel{risk_level}_std', 'app_continunous_rlevel{risk_level}_d{time_window}_max', 'app_continunous_rlevel{risk_level}_d{time_window}_min', 'app_continunous_rlevel{risk_level}_d{time_window}_mean', 'app_continunous_rlevel{risk_level}_d{time_window}_std', 'app_shift_diff_rlevel{risk_level}_d{time_window}', 'app_shift_diff_ratio_rlevel{risk_level}_d{time_window}', 'app_cnt_rlevel{risk_level}_{time_period}_d{time_window}', 'app_ratio_rlevel{risk_level}_{time_period}_d{time_window}']
app_features_df['features_meaning'] = ['逾期率分箱为risk_level的app在time_window天内的数量','逾期率分箱为risk_level的app在time_window天内的占比', '逾期率分箱为risk_level的app在time_window天内距apply_time的最大时间差', '逾期率分箱为risk_level的app在time_window天内距apply_time的最小时间差', '逾期率分箱为risk_level的app在time_window天内距apply_time的平均时间差', '逾期率分箱为risk_level的app在time_window天内距apply_time的时间差的标准差', '逾期率分箱为risk_level的app在time_window天内连续出现的最大天数', '逾期率分箱为risk_level的app在time_window天内连续出现的最小天数', '逾期率分箱为risk_level的app在time_window天内连续出现的平均天数', '逾期率分箱为risk_level的app在time_window天内连续出现的天数的标准差', '逾期率分箱为risk_level的app在time_window天内的变化量', '逾期率分箱为risk_level的app在time_window天内的变化比例', '逾期率分箱为risk_level的app在time_window天内在time_period时间段内的数量', '逾期率分箱为risk_level的app在time_window天内在time_period时间段内的占比']
app_features_df

,feature,features_meaning
0,app_cnt_rlevel{risk_level}_d{time_window},逾期率分箱为risk_level的app在time_window天内的数量
1,app_ratio_rlevel{risk_level}_d{time_window},逾期率分箱为risk_level的app在time_window天内的占比
2,app_time_diff_rlevel{risk_level}_max,逾期率分箱为risk_level的app在time_window天内距apply_time的...
3,app_time_diff_rlevel{risk_level}_min,逾期率分箱为risk_level的app在time_window天内距apply_time的...
4,app_time_diff_rlevel{risk_level}_mean,逾期率分箱为risk_level的app在time_window天内距apply_time的...
5,app_time_diff_rlevel{risk_level}_std,逾期率分箱为risk_level的app在time_window天内距apply_time的...
6,app_continunous_rlevel{risk_level}_d{time_wind...,逾期率分箱为risk_level的app在time_window天内连续出现的最大天数
7,app_continunous_rlevel{risk_level}_d{time_wind...,逾期率分箱为risk_level的app在time_window天内连续出现的最小天数
8,app_continunous_rlevel{risk_level}_d{time_wind...,逾期率分箱为risk_level的app在time_window天内连续出现的平均天数
9,app_continunous_rlevel{risk_level}_d{time_wind...,逾期率分箱为risk_level的app在time_window天内连续出现的天数的标准差


In [36]:
# sender特征
# sender_cnt_rlevel{risk_level}_d{time_window} 逾期率分箱为risk_level的sender在time_window天内的数量
# sender_ratio_rlevel{risk_level}_d{time_window} 逾期率分箱为risk_level的sender在time_window天内的占比
# sender_cnt_rlevel{risk_level}_d{time_window}_max 逾期率分箱为risk_level的sender在time_window天内的最大数量
# sender_cnt_rlevel{risk_level}_d{time_window}_min 逾期率分箱为risk_level的sender在time_window天内的最小数量
# sender_cnt_rlevel{risk_level}_d{time_window}_mean 逾期率分箱为risk_level的sender在time_window天内的平均数量
# sender_cnt_rlevel{risk_level}_d{time_window}_std 逾期率分箱为risk_level的sender在time_window天内的数量的标准差
# sender_time_diff_rlevel{risk_level}_max 逾期率分箱为risk_level的sender在time_window天内距apply_time的最大时间差
# sender_time_diff_rlevel{risk_level}_min 逾期率分箱为risk_level的sender在time_window天内距apply_time的最小时间差
# sender_time_diff_rlevel{risk_level}_mean 逾期率分箱为risk_level的sender在time_window天内距apply_time的平均时间差
# sender_time_diff_rlevel{risk_level}_std 逾期率分箱为risk_level的sender在time_window天内距apply_time的时间差的标准差
# sender_continunous_rlevel{risk_level}_d{time_window}_max 逾期率分箱为risk_level的sender在time_window天内连续出现的最大天数
# sender_continunous_rlevel{risk_level}_d{time_window}_min 逾期率分箱为risk_level的sender在time_window天内连续出现的最小天数
# sender_continunous_rlevel{risk_level}_d{time_window}_mean 逾期率分箱为risk_level的sender在time_window天内连续出现的平均天数
# sender_continunous_rlevel{risk_level}_d{time_window}_std 逾期率分箱为risk_level的sender在time_window天内连续出现的天数的标准差
# sender_shift_diff_rlevel{risk_level}_d{time_window} 逾期率分箱为risk_level的sender在time_window天内的变化量
# sender_shift_diff_ratio_rlevel{risk_level}_d{time_window} 逾期率分箱为risk_level的sender在time_window天内的变化比例
# sender_cnt_rlevel{risk_level}_{time_period}_d{time_window} 逾期率分箱为risk_level的sender在time_window天内在time_period时间段内的数量
# sender_ratio_rlevel{risk_level}_{time_period}_d{time_window} 逾期率分箱为risk_level的sender在time_window天内在time_period时间段内的占比
sender_features_df = pd.DataFrame(columns=['feature', 'features_meaning'])
sender_features_df['feature'] = ['sender_cnt_rlevel{risk_level}_d{time_window}', 'sender_ratio_rlevel{risk_level}_d{time_window}', 'sender_cnt_rlevel{risk_level}_d{time_window}_max', 'sender_cnt_rlevel{risk_level}_d{time_window}_min', 'sender_cnt_rlevel{risk_level}_d{time_window}_mean', 'sender_cnt_rlevel{risk_level}_d{time_window}_std', 'sender_time_diff_rlevel{risk_level}_max', 'sender_time_diff_rlevel{risk_level}_min', 'sender_time_diff_rlevel{risk_level}_mean', 'sender_time_diff_rlevel{risk_level}_std', 'sender_continunous_rlevel{risk_level}_d{time_window}_max', 'sender_continunous_rlevel{risk_level}_d{time_window}_min', 'sender_continunous_rlevel{risk_level}_d{time_window}_mean', 'sender_continunous_rlevel{risk_level}_d{time_window}_std', 'sender_shift_diff_rlevel{risk_level}_d{time_window}', 'sender_shift_diff_ratio_rlevel{risk_level}_d{time_window}', 'sender_cnt_rlevel{risk_level}_{time_period}_d{time_window}', 'sender_ratio_rlevel{risk_level}_{time_period}_d{time_window}']
sender_features_df['features_meaning'] = ['逾期率分箱为risk_level的sender在time_window天内的数量', '逾期率分箱为risk_level的sender在time_window天内的占比', '逾期率分箱为risk_level的sender在time_window天内的最大数量', '逾期率分箱为risk_level的sender在time_window天内的最小数量', '逾期率分箱为risk_level的sender在time_window天内的平均数量', '逾期率分箱为risk_level的sender在time_window天内的数量的标准差', '逾期率分箱为risk_level的sender在time_window天内距apply_time的最大时间差', '逾期率分箱为risk_level的sender在time_window天内距apply_time的最小时间差', '逾期率分箱为risk_level的sender在time_window天内距apply_time的平均时间差', '逾期率分箱为risk_level的sender在time_window天内距apply_time的时间差的标准差', '逾期率分箱为risk_level的sender在time_window天内连续出现的最大天数', '逾期率分箱为risk_level的sender在time_window天内连续出现的最小天数', '逾期率分箱为risk_level的sender在time_window天内连续出现的平均天数', '逾期率分箱为risk_level的sender在time_window天内连续出现的天数的标准差', '逾期率分箱为risk_level的sender在time_window天内的变化量', '逾期率分箱为risk_level的sender在time_window天内的变化比例', '逾期率分箱为risk_level的sender在time_window天内在time_period时间段内的数量', '逾期率分箱为risk_level的sender在time_window天内在time_period时间段内的占比']
sender_features_df

,feature,features_meaning
0,sender_cnt_rlevel{risk_level}_d{time_window},逾期率分箱为risk_level的sender在time_window天内的数量
1,sender_ratio_rlevel{risk_level}_d{time_window},逾期率分箱为risk_level的sender在time_window天内的占比
2,sender_cnt_rlevel{risk_level}_d{time_window}_max,逾期率分箱为risk_level的sender在time_window天内的最大数量
3,sender_cnt_rlevel{risk_level}_d{time_window}_min,逾期率分箱为risk_level的sender在time_window天内的最小数量
4,sender_cnt_rlevel{risk_level}_d{time_window}_mean,逾期率分箱为risk_level的sender在time_window天内的平均数量
5,sender_cnt_rlevel{risk_level}_d{time_window}_std,逾期率分箱为risk_level的sender在time_window天内的数量的标准差
6,sender_time_diff_rlevel{risk_level}_max,逾期率分箱为risk_level的sender在time_window天内距apply_ti...
7,sender_time_diff_rlevel{risk_level}_min,逾期率分箱为risk_level的sender在time_window天内距apply_ti...
8,sender_time_diff_rlevel{risk_level}_mean,逾期率分箱为risk_level的sender在time_window天内距apply_ti...
9,sender_time_diff_rlevel{risk_level}_std,逾期率分箱为risk_level的sender在time_window天内距apply_ti...


In [37]:
# word特征
# word_cnt_rlevel{risk_level}_d{time_window} 逾期率分箱为risk_level的word在time_window天内的数量]
# word_ratio_rlevel{risk_level}_d{time_window} 逾期率分箱为risk_level的word在time_window天内的占比
# word_cnt_rlevel{risk_level}_d{time_window}_max 逾期率分箱为risk_level的word在time_window天内的最大数量
# word_cnt_rlevel{risk_level}_d{time_window}_min 逾期率分箱为risk_level的word在time_window天内的最小数量
# word_cnt_rlevel{risk_level}_d{time_window}_mean 逾期率分箱为risk_level的word在time_window天内的平均数量
# word_cnt_rlevel{risk_level}_d{time_window}_std 逾期率分箱为risk_level的word在time_window天内的数量的标准差
# word_time_diff_rlevel{risk_level}_max 逾期率分箱为risk_level的word在time_window天内距apply_time的最大时间差
# word_time_diff_rlevel{risk_level}_min 逾期率分箱为risk_level的word在time_window天内距apply_time的最小时间差
# word_time_diff_rlevel{risk_level}_mean 逾期率分箱为risk_level的word在time_window天内距apply_time的平均时间差
# word_time_diff_rlevel{risk_level}_std 逾期率分箱为risk_level的word在time_window天内距apply_time的时间差的标准差
# word_continunous_rlevel{risk_level}_d{time_window}_max 逾期率分箱为risk_level的word在time_window天内连续出现的最大天数
# word_continunous_rlevel{risk_level}_d{time_window}_min 逾期率分箱为risk_level的word在time_window天内连续出现的最小天数
# word_continunous_rlevel{risk_level}_d{time_window}_mean 逾期率分箱为risk_level的word在time_window天内连续出现的平均天数
# word_continunous_rlevel{risk_level}_d{time_window}_std 逾期率分箱为risk_level的word在time_window天内连续出现的天数的标准差
# word_shift_diff_rlevel{risk_level}_d{time_window} 逾期率分箱为risk_level的word在time_window天内的变化量
# word_shift_diff_ratio_rlevel{risk_level}_d{time_window} 逾期率分箱为risk_level的word在time_window天内的变化比例
# word_cnt_rlevel{risk_level}_{time_period}_d{time_window} 逾期率分箱为risk_level的word在time_window天内在time_period时间段内的数量
# word_ratio_rlevel{risk_level}_{time_period}_d{time_window} 逾期率分箱为risk_level的word在time_window天内在time_period时间段内的占比
word_features_df = pd.DataFrame(columns=['feature', 'features_meaning'])
word_features_df['feature'] = ['word_cnt_rlevel{risk_level}_d{time_window}', 'word_ratio_rlevel{risk_level}_d{time_window}', 'word_cnt_rlevel{risk_level}_d{time_window}_max', 'word_cnt_rlevel{risk_level}_d{time_window}_min', 'word_cnt_rlevel{risk_level}_d{time_window}_mean', 'word_cnt_rlevel{risk_level}_d{time_window}_std', 'word_time_diff_rlevel{risk_level}_max', 'word_time_diff_rlevel{risk_level}_min', 'word_time_diff_rlevel{risk_level}_mean', 'word_time_diff_rlevel{risk_level}_std', 'word_continunous_rlevel{risk_level}_d{time_window}_max', 'word_continunous_rlevel{risk_level}_d{time_window}_min', 'word_continunous_rlevel{risk_level}_d{time_window}_mean', 'word_continunous_rlevel{risk_level}_d{time_window}_std', 'word_shift_diff_rlevel{risk_level}_d{time_window}', 'word_shift_diff_ratio_rlevel{risk_level}_d{time_window}', 'word_cnt_rlevel{risk_level}_{time_period}_d{time_window}', 'word_ratio_rlevel{risk_level}_{time_period}_d{time_window}']
word_features_df['features_meaning'] = ['逾期率分箱为risk_level的word在time_window天内的数量', '逾期率分箱为risk_level的word在time_window天内的占比', '逾期率分箱为risk_level的word在time_window天内的最大数量', '逾期率分箱为risk_level的word在time_window天内的最小数量', '逾期率分箱为risk_level的word在time_window天内的平均数量', '逾期率分箱为risk_level的word在time_window天内的数量的标准差', '逾期率分箱为risk_level的word在time_window天内距apply_time的最大时间差', '逾期率分箱为risk_level的word在time_window天内距apply_time的最小时间差', '逾期率分箱为risk_level的word在time_window天内距apply_time的平均时间差', '逾期率分箱为risk_level的word在time_window天内距apply_time的时间差的标准差', '逾期率分箱为risk_level的word在time_window天内连续出现的最大天数', '逾期率分箱为risk_level的word在time_window天内连续出现的最小天数', '逾期率分箱为risk_level的word在time_window天内连续出现的平均天数', '逾期率分箱为risk_level的word在time_window天内连续出现的天数的标准差', '逾期率分箱为risk_level的word在time_window天内的变化量', '逾期率分箱为risk_level的word在time_window天内的变化比例', '逾期率分箱为risk_level的word在time_window天内在time_period时间段内的数量', '逾期率分箱为risk_level的word在time_window天内在time_period时间段内的占比']
word_features_df


,feature,features_meaning
0,word_cnt_rlevel{risk_level}_d{time_window},逾期率分箱为risk_level的word在time_window天内的数量
1,word_ratio_rlevel{risk_level}_d{time_window},逾期率分箱为risk_level的word在time_window天内的占比
2,word_cnt_rlevel{risk_level}_d{time_window}_max,逾期率分箱为risk_level的word在time_window天内的最大数量
3,word_cnt_rlevel{risk_level}_d{time_window}_min,逾期率分箱为risk_level的word在time_window天内的最小数量
4,word_cnt_rlevel{risk_level}_d{time_window}_mean,逾期率分箱为risk_level的word在time_window天内的平均数量
5,word_cnt_rlevel{risk_level}_d{time_window}_std,逾期率分箱为risk_level的word在time_window天内的数量的标准差
6,word_time_diff_rlevel{risk_level}_max,逾期率分箱为risk_level的word在time_window天内距apply_time...
7,word_time_diff_rlevel{risk_level}_min,逾期率分箱为risk_level的word在time_window天内距apply_time...
8,word_time_diff_rlevel{risk_level}_mean,逾期率分箱为risk_level的word在time_window天内距apply_time...
9,word_time_diff_rlevel{risk_level}_std,逾期率分箱为risk_level的word在time_window天内距apply_time...


In [38]:
# 将app、sender、word的特征含义保存到一个excel的不同sheet中
with pd.ExcelWriter('features_meaning.xlsx') as writer:
    app_features_df.to_excel(writer, sheet_name='app_features')
    sender_features_df.to_excel(writer, sheet_name='sender_features')
    word_features_df.to_excel(writer, sheet_name='word_features')
    